In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './한국부동산원_주택가격지수/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.xlsx')]
filenames

['월간_매매가격지수_아파트.xlsx',
 '월간_매매가격지수_연립다세대.xlsx',
 '월간_매매가격지수_종합.xlsx',
 '월간_전세가격지수_아파트.xlsx',
 '월간_전세가격지수_연립다세대.xlsx',
 '월간_전세가격지수_종합.xlsx']

In [4]:
for file in filenames:
    f = open(basedir + file, 'r')
    try:
        print(f.readline())
    except:
        f.close()
        pass
    f.close()

In [20]:
apart_trade_price_index_df = pd.read_excel(basedir + filenames[0], header=[10,11], index_col=[0,1,2])
print(apart_trade_price_index_df.shape)
apart_trade_price_index_df.head()

(204, 360)


지 역        2006년 01월       2006년 02월       2006년 03월       2006년 04월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    66.277   0.8    66.924  0.98    68.017  1.63    69.468  2.13   
   종로구 NaN    68.522     0    68.991  0.69    69.909  1.33    69.967  0.08   
   중구  NaN    73.997   0.7    74.145   0.2    75.042  1.21    75.854  1.08   
   용산구 NaN    72.035  1.14    73.594  2.16    74.815  1.66    78.664  5.14   
   성동구 NaN    68.886  0.51    69.837  1.38    70.607   1.1    71.577  1.37   

지 역        2006년 05월       2006년 06월       2006년 07월       2006년 08월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN     70.81  1.93    71.302  0.69    71.524  0.31    71.741   0.3   
   종로구 NaN    69.973  0.01    70.202  0.33     69.98 -0.32    70.154  0.25   
   중구  NaN    76.154   0.4    77.007  1.12     77.57  0.73    77.859  0.37   
   용산구 NaN    81.348  3.41    82.683  1.64    83.228  0.66    83.517  0.35   
   성동구 NaN    72.802  1.71    73.759  1.31    74.434  0.92    75.108  0.91   

지 역        2006년 09월       2006년 10월       2006년 11월       2006년 12월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    72.277  0.75    73.843  2.17    78.198   5.9    81.176  3.81   
   종로구 NaN    70.826  0.96    71.817   1.4    74.949  4.36    76.367  1.89   
   중구  NaN    78.308  0.58    81.524  4.11    83.924  2.94    86.767  3.39   
   용산구 NaN     83.87  0.42    85.431  1.86    88.897  4.06    91.576  3.01   
   성동구 NaN    75.287  0.24    77.075  2.37    81.174  5.32    84.292  3.84   

지 역        2007년 01월       2007년 02월       2007년 03월       2007년 04월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    83.058  2.32     83.63  0.69    83.952  0.38    84.267  0.37   
   종로구 NaN    77.416  1.37    77.988  0.74     78.26  0.35    78.437  0.23   
   중구  NaN    88.307  1.77     89.14  0.94    89.483  0.38    89.567  0.09   
   용산구 NaN    92.991  1.55    93.356  0.39    93.502  0.16    93.965   0.5   
   성동구 NaN    85.664  1.63    87.147  1.73    87.595  0.51    88.054  0.52   

지 역        2007년 05월       2007년 06월       2007년 07월       2007년 08월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    84.272  0.01    84.484  0.25    84.849  0.43    85.154  0.36   
   종로구 NaN    78.555  0.15    79.049  0.63    79.325  0.35      79.4  0.09   
   중구  NaN    90.095  0.59     90.18  0.09    90.747  0.63    90.897  0.17   
   용산구 NaN    93.889 -0.08    95.221  1.42    95.674  0.48    96.158  0.51   
   성동구 NaN    88.138   0.1    88.428  0.33    88.469  0.05    88.573  0.12   

지 역        2007년 09월       2007년 10월       2007년 11월       2007년 12월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    85.559  0.47    85.933  0.44    86.516  0.68    86.863   0.4   
   종로구 NaN     79.66  0.33     79.83  0.21    79.947  0.15    80.471  0.66   
   중구  NaN    91.481  0.64    92.212   0.8    92.187 -0.03    92.501  0.34   
   용산구 NaN     96.73  0.59    97.659  0.96    98.473  0.83     98.94  0.47   
   성동구 NaN    88.901  0.37     88.95  0.06    89.313  0.41    89.662  0.39   

지 역        2008년 01월       2008년 02월       2008년 03월       2008년 04월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    87.503  0.74    88.134  0.72    89.707  1.78    91.947   2.5   
   종로구 NaN    81.077  0.75    81.469  0.48    82.194  0.89    83.856  2.02   
   중구  NaN    93.708   1.3    94.116  0.43    94.719  0.64    95.944  1.29   
   용산구 NaN    99.542  0.61   100.597  1.06    101.74  1.14   103.253  1.49   
   성동구 NaN    89.574  -0.1    89.777  0.23    91.002  1.36    91.715  0.78   

지 역        2008년 05월       2008년 06월       2008년 07월       2008년 08월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    9

In [56]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_2.csv')
print(main_data_df.shape)
main_data_df.head()

(4441256, 17)


,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표
0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
2,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
3,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
4,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06


In [57]:
main_data_df['주택가격지수'] = np.nan
main_data_df['주택가격지수'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: 주택가격지수, dtype: float64

In [58]:
apart_trade_df = main_data_df[(main_data_df['건물종류'] == '아파트')&(main_data_df['전월세매매구분'] == '매매')]
apart_trade_df.shape

(1093235, 18)

In [59]:
def get_gu(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        return x.split(' ')[1]

In [60]:
apart_trade_df['구'] = apart_trade_df['지번주소'].apply(get_gu)
apart_trade_df['구'].value_counts()

노원구     115209
송파구      72626
강서구      67960
강남구      64521
구로구      58364
양천구      56292
성북구      55336
강동구      53166
도봉구      51774
서초구      48917
영등포구     47149
마포구      40354
동작구      39668
성동구      38642
동대문구     38512
중랑구      36303
관악구      36285
은평구      36149
서대문구     31706
광진구      22259
강북구      21192
용산구      20341
금천구      19106
중구       12380
종로구       8801
Name: 구, dtype: int64

In [29]:
apart_trade_price_index_df.columns

MultiIndex([('2006년 01월',  '지수'),
            ('2006년 01월', '변동률'),
            ('2006년 02월',  '지수'),
            ('2006년 02월', '변동률'),
            ('2006년 03월',  '지수'),
            ('2006년 03월', '변동률'),
            ('2006년 04월',  '지수'),
            ('2006년 04월', '변동률'),
            ('2006년 05월',  '지수'),
            ('2006년 05월', '변동률'),
            ...
            ('2020년 08월',  '지수'),
            ('2020년 08월', '변동률'),
            ('2020년 09월',  '지수'),
            ('2020년 09월', '변동률'),
            ('2020년 10월',  '지수'),
            ('2020년 10월', '변동률'),
            ('2020년 11월',  '지수'),
            ('2020년 11월', '변동률'),
            ('2020년 12월',  '지수'),
            ('2020년 12월', '변동률')],
           names=['지 역', None], length=360)

In [31]:
cols_to_drop = []
for col in apart_trade_price_index_df.columns:
    if '변동률' in col:
        cols_to_drop.append(col)
cols_to_drop

[('2006년 01월', '변동률'),
 ('2006년 02월', '변동률'),
 ('2006년 03월', '변동률'),
 ('2006년 04월', '변동률'),
 ('2006년 05월', '변동률'),
 ('2006년 06월', '변동률'),
 ('2006년 07월', '변동률'),
 ('2006년 08월', '변동률'),
 ('2006년 09월', '변동률'),
 ('2006년 10월', '변동률'),
 ('2006년 11월', '변동률'),
 ('2006년 12월', '변동률'),
 ('2007년 01월', '변동률'),
 ('2007년 02월', '변동률'),
 ('2007년 03월', '변동률'),
 ('2007년 04월', '변동률'),
 ('2007년 05월', '변동률'),
 ('2007년 06월', '변동률'),
 ('2007년 07월', '변동률'),
 ('2007년 08월', '변동률'),
 ('2007년 09월', '변동률'),
 ('2007년 10월', '변동률'),
 ('2007년 11월', '변동률'),
 ('2007년 12월', '변동률'),
 ('2008년 01월', '변동률'),
 ('2008년 02월', '변동률'),
 ('2008년 03월', '변동률'),
 ('2008년 04월', '변동률'),
 ('2008년 05월', '변동률'),
 ('2008년 06월', '변동률'),
 ('2008년 07월', '변동률'),
 ('2008년 08월', '변동률'),
 ('2008년 09월', '변동률'),
 ('2008년 10월', '변동률'),
 ('2008년 11월', '변동률'),
 ('2008년 12월', '변동률'),
 ('2009년 01월', '변동률'),
 ('2009년 02월', '변동률'),
 ('2009년 03월', '변동률'),
 ('2009년 04월', '변동률'),
 ('2009년 05월', '변동률'),
 ('2009년 06월', '변동률'),
 ('2009년 07월', '변동률'),
 ('2009년 08

In [32]:
apart_trade_price_index_df.drop(columns=cols_to_drop, inplace=True)
apart_trade_price_index_df.head()

지 역        2006년 01월 2006년 02월 2006년 03월 2006년 04월 2006년 05월 2006년 06월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    66.277    66.924    68.017    69.468     70.81    71.302   
   종로구 NaN    68.522    68.991    69.909    69.967    69.973    70.202   
   중구  NaN    73.997    74.145    75.042    75.854    76.154    77.007   
   용산구 NaN    72.035    73.594    74.815    78.664    81.348    82.683   
   성동구 NaN    68.886    69.837    70.607    71.577    72.802    73.759   

지 역        2006년 07월 2006년 08월 2006년 09월 2006년 10월 2006년 11월 2006년 12월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    71.524    71.741    72.277    73.843    78.198    81.176   
   종로구 NaN     69.98    70.154    70.826    71.817    74.949    76.367   
   중구  NaN     77.57    77.859    78.308    81.524    83.924    86.767   
   용산구 NaN    83.228    83.517     83.87    85.431    88.897    91.576   
   성동구 NaN    74.434    75.108    75.287    77.075    81.174    84.292   

지 역        2007년 01월 2007년 02월 2007년 03월 2007년 04월 2007년 05월 2007년 06월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    83.058     83.63    83.952    84.267    84.272    84.484   
   종로구 NaN    77.416    77.988     78.26    78.437    78.555    79.049   
   중구  NaN    88.307     89.14    89.483    89.567    90.095     90.18   
   용산구 NaN    92.991    93.356    93.502    93.965    93.889    95.221   
   성동구 NaN    85.664    87.147    87.595    88.054    88.138    88.428   

지 역        2007년 07월 2007년 08월 2007년 09월 2007년 10월 2007년 11월 2007년 12월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    84.849    85.154    85.559    85.933    86.516    86.863   
   종로구 NaN    79.325      79.4     79.66     79.83    79.947    80.471   
   중구  NaN    90.747    90.897    91.481    92.212    92.187    92.501   
   용산구 NaN    95.674    96.158     96.73    97.659    98.473     98.94   
   성동구 NaN    88.469    88.573    88.901     88.95    89.313    89.662   

지 역        2008년 01월 2008년 02월 2008년 03월 2008년 04월 2008년 05월 2008년 06월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    87.503    88.134    89.707    91.947    93.335    94.673   
   종로구 NaN    81.077    81.469    82.194    83.856    84.012    84.458   
   중구  NaN    93.708    94.116    94.719    95.944     97.07    97.648   
   용산구 NaN    99.542   100.597    101.74   103.253   103.888   104.675   
   성동구 NaN    89.574    89.777    91.002    91.715    92.283     92.55   

지 역        2008년 07월 2008년 08월 2008년 09월 2008년 10월 2008년 11월 2008년 12월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    95.393    95.613    95.831    95.541    94.687    93.048   
   종로구 NaN    85.601    86.206    87.312    87.815    89.135    88.822   
   중구  NaN     97.67    97.744    97.744    97.515    96.817    96.103   
   용산구 NaN   105.768   106.075   106.379   106.591   105.519    103.24   
   성동구 NaN    93.115    93.116    93.489    93.438    93.264    92.513   

지 역        2009년 01월 2009년 02월 2009년 03월 2009년 04월 2009년 05월 2009년 06월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    92.197    92.028    91.787    92.095    92.287    92.678   
   종로구 NaN    88.747    88.545    89.003    89.937     90.03    90.275   
   중구  NaN    95.248    95.738    96.255    96.392    96.847    97.288   
   용산구 NaN   102.247   101.964   101.515   101.531   101.675   101.997   
   성동구 NaN    92.154    91.933    91.875     92.11    92.054     92.12   

지 역        2009년 07월 2009년 08월 2009년 09월 2009년 10월 2009년 11월 2009년 12월  \
                  지수        지수        지수        지수        지수        지수   
서울 NaN NaN    93.466     93.95    95.016    95.328    95.336    95.334   
   종로구 NaN    90.193    90.511    91.709    92.425    92.311    92.399   
   중구  NaN    97.696    97.696    98.712    98.751    98.524    98.524 

In [33]:
apart_trade_price_index_df.columns = [col[0] for col in apart_trade_price_index_df.columns]
apart_trade_price_index_df.head()

2006년 01월 2006년 02월 2006년 03월 2006년 04월 2006년 05월 2006년 06월  \
서울 NaN NaN    66.277    66.924    68.017    69.468     70.81    71.302   
   종로구 NaN    68.522    68.991    69.909    69.967    69.973    70.202   
   중구  NaN    73.997    74.145    75.042    75.854    76.154    77.007   
   용산구 NaN    72.035    73.594    74.815    78.664    81.348    82.683   
   성동구 NaN    68.886    69.837    70.607    71.577    72.802    73.759   

           2006년 07월 2006년 08월 2006년 09월 2006년 10월 2006년 11월 2006년 12월  \
서울 NaN NaN    71.524    71.741    72.277    73.843    78.198    81.176   
   종로구 NaN     69.98    70.154    70.826    71.817    74.949    76.367   
   중구  NaN     77.57    77.859    78.308    81.524    83.924    86.767   
   용산구 NaN    83.228    83.517     83.87    85.431    88.897    91.576   
   성동구 NaN    74.434    75.108    75.287    77.075    81.174    84.292   

           2007년 01월 2007년 02월 2007년 03월 2007년 04월 2007년 05월 2007년 06월  \
서울 NaN NaN    83.058     83.63    83.952    84.267    84.272    84.484   
   종로구 NaN    77.416    77.988     78.26    78.437    78.555    79.049   
   중구  NaN    88.307     89.14    89.483    89.567    90.095     90.18   
   용산구 NaN    92.991    93.356    93.502    93.965    93.889    95.221   
   성동구 NaN    85.664    87.147    87.595    88.054    88.138    88.428   

           2007년 07월 2007년 08월 2007년 09월 2007년 10월 2007년 11월 2007년 12월  \
서울 NaN NaN    84.849    85.154    85.559    85.933    86.516    86.863   
   종로구 NaN    79.325      79.4     79.66     79.83    79.947    80.471   
   중구  NaN    90.747    90.897    91.481    92.212    92.187    92.501   
   용산구 NaN    95.674    96.158     96.73    97.659    98.473     98.94   
   성동구 NaN    88.469    88.573    88.901     88.95    89.313    89.662   

           2008년 01월 2008년 02월 2008년 03월 2008년 04월 2008년 05월 2008년 06월  \
서울 NaN NaN    87.503    88.134    89.707    91.947    93.335    94.673   
   종로구 NaN    81.077    81.469    82.194    83.856    84.012    84.458   
   중구  NaN    93.708    94.116    94.719    95.944     97.07    97.648   
   용산구 NaN    99.542   100.597    101.74   103.253   103.888   104.675   
   성동구 NaN    89.574    89.777    91.002    91.715    92.283     92.55   

           2008년 07월 2008년 08월 2008년 09월 2008년 10월 2008년 11월 2008년 12월  \
서울 NaN NaN    95.393    95.613    95.831    95.541    94.687    93.048   
   종로구 NaN    85.601    86.206    87.312    87.815    89.135    88.822   
   중구  NaN     97.67    97.744    97.744    97.515    96.817    96.103   
   용산구 NaN   105.768   106.075   106.379   106.591   105.519    103.24   
   성동구 NaN    93.115    93.116    93.489    93.438    93.264    92.513   

           2009년 01월 2009년 02월 2009년 03월 2009년 04월 2009년 05월 2009년 06월  \
서울 NaN NaN    92.197    92.028    91.787    92.095    92.287    92.678   
   종로구 NaN    88.747    88.545    89.003    89.937     90.03    90.275   
   중구  NaN    95.248    95.738    96.255    96.392    96.847    97.288   
   용산구 NaN   102.247   101.964   101.515   101.531   101.675   101.997   
   성동구 NaN    92.154    91.933    91.875     92.11    92.054     92.12   

           2009년 07월 2009년 08월 2009년 09월 2009년 10월 2009년 11월 2009년 12월  \
서울 NaN NaN    93.466     93.95    95.016    95.328    95.336    95.334   
   종로구 NaN    90.193    90.511    91.709    92.425    92.311    92.399   
   중구  NaN    97.696    97.696    98.712    98.751    98.524    98.524   
   용산구 NaN   102.663   102.844   103.768   104.114   104.404    104.58   
   성동구 NaN    92.129    92.177    92.485    92.498    92.564    92.569   

           2010년 01월 2010년 02월 2010년 03월 2010년 04월 2010년 05월 2010년 06월  \
서울 NaN NaN    95.411    95.656    95.641    95.544    95.112    94.751   
   종로구 NaN    92.556    92.716    92.786    92.712    92.647    92.673   
   중구  NaN    98.477    98.958    99.159    99.101    98.917    98.768   
   용산구 NaN   104.603   104.684   104.811   104.907   104.808   104.308   
   성동구 NaN    92.588    92.711    92.715    92.691     92.39    92.526   

      

In [36]:
apart_trade_price_index_df = apart_trade_price_index_df.loc['서울']

In [38]:
apart_trade_price_index_df.shape

(26, 180)

In [39]:
apart_trade_price_index_df.head()

,,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
NaN,NaN,66.277,66.924,68.017,69.468,70.81,71.302,71.524,71.741,72.277,73.843,78.198,81.176,83.058,83.63,83.952,84.267,84.272,84.484,84.849,85.154,85.559,85.933,86.516,86.863,87.503,88.134,89.707,91.947,93.335,94.673,95.393,95.613,95.831,95.541,94.687,93.048,92.197,92.028,91.787,92.095,92.287,92.678,93.466,93.95,95.016,95.328,95.336,95.334,95.411,95.656,95.641,95.544,95.112,94.751,94.293,93.857,93.619,93.441,93.329,93.37,93.501,93.761,93.891,93.901,93.933,93.823,93.672,93.584,93.47,93.347,93.175,93.014,92.762,92.527,92.127,91.605,91.108,90.389,89.778,88.973,88.379,87.83,87.388,86.829,86.286,85.952,85.847,85.964,86.167,85.977,85.52,85.115,85.192,85.643,85.747,85.72,85.97,86.263,86.602,86.632,86.557,86.496,86.473,86.573,86.92,87.29,87.413,87.425,87.547,87.817,88.27,88.938,89.469,90.078,90.565,91.068,91.685,92.303,92.899,93.29,93.333,93.333,93.328,93.419,93.616,93.934,94.259,94.625,94.998,95.653,96.192,96.318,96.342,96.407,96.566,96.832,97.27,98.237,98.73,99.33,99.32,99.577,100,100.837,102.19,103.606,104.4,104.789,105.023,105.249,105.607,106.474,108.437,109.066,109.123,108.939,108.493,108.094,107.665,107.296,107.082,106.984,107.06,107.212,107.409,108.052,108.8,110.146,110.639,110.769,110.883,110.776,110.55,110.697,111.939,112.551,112.883,113.011,113.144,113.463
종로구,NaN,68.522,68.991,69.909,69.967,69.973,70.202,69.98,70.154,70.826,71.817,74.949,76.367,77.416,77.988,78.26,78.437,78.555,79.049,79.325,79.4,79.66,79.83,79.947,80.471,81.077,81.469,82.194,83.856,84.012,84.458,85.601,86.206,87.312,87.815,89.135,88.822,88.747,88.545,89.003,89.937,90.03,90.275,90.193,90.511,91.709,92.425,92.311,92.399,92.556,92.716,92.786,92.712,92.647,92.673,92.453,92.118,92.08,92.039,92.074,92.011,92.011,92.104,92.656,92.661,92.537,92.636,92.522,93.044,93.044,93.052,92.961,92.928,92.928,92.928,92.928,92.928,92.928,92.879,92.879,93.012,92.939,92.672,92.617,92.613,92.613,92.613,92.533,92.506,92.424,92.116,91.905,91.724,91.689,91.875,91.943,91.987,92.004,92.115,92.125,92.066,92.02,91.951,91.931,91.95,92.196,92.237,92.235,92.201,92.222,92.251,92.472,92.904,93.168,93.399,93.571,93.752,93.917,94.065,94.337,94.441,94.487,94.501,94.531,94.581,94.723,94.9,95.162,95.486,95.675,96.001,96.362,96.548,

In [42]:
idx_list = []
for i in range(len(apart_trade_price_index_df.index)):
    if i == 0:
        idx_list.append('서울전체')
    else:
        idx_list.append(apart_trade_price_index_df.index[i][0])
idx_list

['서울전체',
 '종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [43]:
apart_trade_price_index_df.index = idx_list
apart_trade_price_index_df.head()

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,66.277,66.924,68.017,69.468,70.81,71.302,71.524,71.741,72.277,73.843,78.198,81.176,83.058,83.63,83.952,84.267,84.272,84.484,84.849,85.154,85.559,85.933,86.516,86.863,87.503,88.134,89.707,91.947,93.335,94.673,95.393,95.613,95.831,95.541,94.687,93.048,92.197,92.028,91.787,92.095,92.287,92.678,93.466,93.95,95.016,95.328,95.336,95.334,95.411,95.656,95.641,95.544,95.112,94.751,94.293,93.857,93.619,93.441,93.329,93.37,93.501,93.761,93.891,93.901,93.933,93.823,93.672,93.584,93.47,93.347,93.175,93.014,92.762,92.527,92.127,91.605,91.108,90.389,89.778,88.973,88.379,87.83,87.388,86.829,86.286,85.952,85.847,85.964,86.167,85.977,85.52,85.115,85.192,85.643,85.747,85.72,85.97,86.263,86.602,86.632,86.557,86.496,86.473,86.573,86.92,87.29,87.413,87.425,87.547,87.817,88.27,88.938,89.469,90.078,90.565,91.068,91.685,92.303,92.899,93.29,93.333,93.333,93.328,93.419,93.616,93.934,94.259,94.625,94.998,95.653,96.192,96.318,96.342,96.407,96.566,96.832,97.27,98.237,98.73,99.33,99.32,99.577,100,100.837,102.19,103.606,104.4,104.789,105.023,105.249,105.607,106.474,108.437,109.066,109.123,108.939,108.493,108.094,107.665,107.296,107.082,106.984,107.06,107.212,107.409,108.052,108.8,110.146,110.639,110.769,110.883,110.776,110.55,110.697,111.939,112.551,112.883,113.011,113.144,113.463
종로구,68.522,68.991,69.909,69.967,69.973,70.202,69.98,70.154,70.826,71.817,74.949,76.367,77.416,77.988,78.26,78.437,78.555,79.049,79.325,79.4,79.66,79.83,79.947,80.471,81.077,81.469,82.194,83.856,84.012,84.458,85.601,86.206,87.312,87.815,89.135,88.822,88.747,88.545,89.003,89.937,90.03,90.275,90.193,90.511,91.709,92.425,92.311,92.399,92.556,92.716,92.786,92.712,92.647,92.673,92.453,92.118,92.08,92.039,92.074,92.011,92.011,92.104,92.656,92.661,92.537,92.636,92.522,93.044,93.044,93.052,92.961,92.928,92.928,92.928,92.928,92.928,92.928,92.879,92.879,93.012,92.939,92.672,92.617,92.613,92.613,92.613,92.533,92.506,92.424,92.116,91.905,91.724,91.689,91.875,91.943,91.987,92.004,92.115,92.125,92.066,92.02,91.951,91.931,91.95,92.196,92.237,92.235,92.201,92.222,92.251,92.472,92.904,93.168,93.399,93.571,93.752,93.917,94.065,94.337,94.441,94.487,94.501,94.531,94.581,94.723,94.9,95.162,95.486,95.675,96.001,96.362,96.548,96.68,96

In [44]:
apart_trade_price_index_df

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,66.277,66.924,68.017,69.468,70.81,71.302,71.524,71.741,72.277,73.843,78.198,81.176,83.058,83.63,83.952,84.267,84.272,84.484,84.849,85.154,85.559,85.933,86.516,86.863,87.503,88.134,89.707,91.947,93.335,94.673,95.393,95.613,95.831,95.541,94.687,93.048,92.197,92.028,91.787,92.095,92.287,92.678,93.466,93.95,95.016,95.328,95.336,95.334,95.411,95.656,95.641,95.544,95.112,94.751,94.293,93.857,93.619,93.441,93.329,93.37,93.501,93.761,93.891,93.901,93.933,93.823,93.672,93.584,93.47,93.347,93.175,93.014,92.762,92.527,92.127,91.605,91.108,90.389,89.778,88.973,88.379,87.83,87.388,86.829,86.286,85.952,85.847,85.964,86.167,85.977,85.52,85.115,85.192,85.643,85.747,85.72,85.97,86.263,86.602,86.632,86.557,86.496,86.473,86.573,86.92,87.29,87.413,87.425,87.547,87.817,88.27,88.938,89.469,90.078,90.565,91.068,91.685,92.303,92.899,93.29,93.333,93.333,93.328,93.419,93.616,93.934,94.259,94.625,94.998,95.653,96.192,96.318,96.342,96.407,96.566,96.832,97.27,98.237,98.73,99.33,99.32,99.577,100,100.837,102.19,103.606,104.4,104.789,105.023,105.249,105.607,106.474,108.437,109.066,109.123,108.939,108.493,108.094,107.665,107.296,107.082,106.984,107.06,107.212,107.409,108.052,108.8,110.146,110.639,110.769,110.883,110.776,110.55,110.697,111.939,112.551,112.883,113.011,113.144,113.463
종로구,68.522,68.991,69.909,69.967,69.973,70.202,69.98,70.154,70.826,71.817,74.949,76.367,77.416,77.988,78.26,78.437,78.555,79.049,79.325,79.4,79.66,79.83,79.947,80.471,81.077,81.469,82.194,83.856,84.012,84.458,85.601,86.206,87.312,87.815,89.135,88.822,88.747,88.545,89.003,89.937,90.03,90.275,90.193,90.511,91.709,92.425,92.311,92.399,92.556,92.716,92.786,92.712,92.647,92.673,92.453,92.118,92.08,92.039,92.074,92.011,92.011,92.104,92.656,92.661,92.537,92.636,92.522,93.044,93.044,93.052,92.961,92.928,92.928,92.928,92.928,92.928,92.928,92.879,92.879,93.012,92.939,92.672,92.617,92.613,92.613,92.613,92.533,92.506,92.424,92.116,91.905,91.724,91.689,91.875,91.943,91.987,92.004,92.115,92.125,92.066,92.02,91.951,91.931,91.95,92.196,92.237,92.235,92.201,92.222,92.251,92.472,92.904,93.168,93.399,93.571,93.752,93.917,94.065,94.337,94.441,94.487,94.501,94.531,94.581,94.723,94.9,95.162,95.486,95.675,96.001,96.362,96.548,96.68,96

In [45]:
main_data_df.head()

,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표,주택가격지수
0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,0
1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,0
2,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,0
3,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,0
4,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,0


In [61]:
for gu in tqdm(apart_trade_df['구'].unique()):
    if pd.isna(gu) == True:
        continue
    gu_df = apart_trade_df[apart_trade_df['구'] == gu]
    for col in apart_trade_price_index_df.columns:
        splitted = col.split(' ')
        year = int(splitted[0].replace('년',''))
        month = int(splitted[1].replace('월',''))
        date_df = gu_df[(gu_df['년'] == year)&(gu_df['월'] == month)]
        date_df['주택가격지수'] = apart_trade_price_index_df.loc[gu, col]
        main_data_df.loc[date_df.index, '주택가격지수'] = date_df['주택가격지수']

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:11<00:00,  2.76s/it]


In [62]:
main_data_df['주택가격지수'].describe()

count    1.093012e+06
mean     9.250434e+01
std      1.075342e+01
min      5.240100e+01
25%      8.722900e+01
50%      9.261200e+01
75%      9.835700e+01
max      1.181710e+02
Name: 주택가격지수, dtype: float64

In [63]:
main_data_df['주택가격지수'].isna().sum()

3348244

In [64]:
filenames

['월간_매매가격지수_아파트.xlsx',
 '월간_매매가격지수_연립다세대.xlsx',
 '월간_매매가격지수_종합.xlsx',
 '월간_전세가격지수_아파트.xlsx',
 '월간_전세가격지수_연립다세대.xlsx',
 '월간_전세가격지수_종합.xlsx']

In [97]:
yunrip_trade_price_index_df = pd.read_excel(basedir + filenames[1], header=[10], index_col=[0,1,2])
print(yunrip_trade_price_index_df.shape)
yunrip_trade_price_index_df.head()

(8, 180)


2006년 01월 2006년 02월 2006년 03월 2006년 04월 2006년 05월 2006년 06월  \
지 역                                                                        
서울  NaN  NaN   64.8385   65.1394   65.4442   65.8232   66.3765    66.971   
    강북지역 NaN    64.265   64.3417   64.5677   64.8161   64.9957   65.3781   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2006년 07월 2006년 08월 2006년 09월 2006년 10월 2006년 11월 2006년 12월  \
지 역                                                                        
서울  NaN  NaN   67.2219   67.4802   68.4117   69.8208   72.1323   73.6332   
    강북지역 NaN   65.7757   65.9864   67.1408   68.6802   70.7498   72.2638   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2007년 01월 2007년 02월 2007년 03월 2007년 04월 2007년 05월 2007년 06월  \
지 역                                                                        
서울  NaN  NaN   75.2736   76.2046   76.9329    77.742   78.1654   78.7278   
    강북지역 NaN   73.6366   74.4338   75.3608   76.1198   76.6601   77.1466   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2007년 07월 2007년 08월 2007년 09월 2007년 10월 2007년 11월 2007년 12월  \
지 역                                                                        
서울  NaN  NaN   79.7804    80.489   81.8179   82.9367   83.7235   84.4694   
    강북지역 NaN    78.147   78.7437   79.7394   80.6139    81.453   82.4994   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2008년 01월 2008년 02월 2008년 03월 2008년 04월 2008년 05월 2008년 06월  \
지 역                                                                        
서울  NaN  NaN   85.8118   86.8334   89.3438   90.9571   92.5835   94.3929   
    강북지역 NaN   84.3305   85.1741   87.7813   89.1495   91.1308   93.4676   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2008년 07월 2008년 08월 2008년 09월 2008년 10월 2008년 11월 2008년 12월  \
지 역                                                                        
서울  NaN  NaN   95.5282   96.3193   97.0417   97.3262   96.8324     95.59   
    강북지역 NaN   94.4015   94.7741   96.0298   96.4586   95.6234   94.7362   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2009년 01월 2009년 02월 2009년 03월 2009년 04월 2009년 05월 2009년 06월  \
지 역                                                                        
서울  NaN  NaN   95.0407   94.9244    94.809   94.7801   94.8022   95.0276   
    강북지역 NaN   94.3421   94.3451   94.2515   94.3572   94.4724   94.6714   
         도심권         -         -         -         -         -         -   
         동북권         -         -         -         -         -         -   
         서북권         -         -         -         -         -         -   

             2009년 07월 2009년 08월 2009년 09월 2009년 10월 2009년 11월 2009년 12월  \
지 역                                                                        
서울  NaN  NaN    95.668   96.1965    97.316   97.9983    98.323   98.4087   
    강북지역 NaN   95.4413    95.941   96.8516   97.5147  

In [98]:
yunrip_trade_price_index_df = yunrip_trade_price_index_df.loc['서울']
yunrip_trade_price_index_df

2006년 01월 2006년 02월 2006년 03월 2006년 04월 2006년 05월 2006년 06월  \
NaN  NaN   64.8385   65.1394   65.4442   65.8232   66.3765    66.971   
강북지역 NaN    64.265   64.3417   64.5677   64.8161   64.9957   65.3781   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   
강남지역 서북권   65.4288   65.9702     66.36   66.8802   67.8409   68.6683   
     서남권         -         -         -         -         -         -   
     동남권         -         -         -         -         -         -   

         2006년 07월 2006년 08월 2006년 09월 2006년 10월 2006년 11월 2006년 12월  \
NaN  NaN   67.2219   67.4802   68.4117   69.8208   72.1323   73.6332   
강북지역 NaN   65.7757   65.9864   67.1408   68.6802   70.7498   72.2638   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   
강남지역 서북권   68.7574   69.0679   69.7545   71.0207   73.5944     75.08   
     서남권         -         -         -         -         -         -   
     동남권         -         -         -         -         -         -   

         2007년 01월 2007년 02월 2007년 03월 2007년 04월 2007년 05월 2007년 06월  \
NaN  NaN   75.2736   76.2046   76.9329    77.742   78.1654   78.7278   
강북지역 NaN   73.6366   74.4338   75.3608   76.1198   76.6601   77.1466   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   
강남지역 서북권   77.0125   78.0901   78.5991   79.4625   79.7577   80.4028   
     서남권         -         -         -         -         -         -   
     동남권         -         -         -         -         -         -   

         2007년 07월 2007년 08월 2007년 09월 2007년 10월 2007년 11월 2007년 12월  \
NaN  NaN   79.7804    80.489   81.8179   82.9367   83.7235   84.4694   
강북지역 NaN    78.147   78.7437   79.7394   80.6139    81.453   82.4994   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   
강남지역 서북권   81.5118   82.3427   84.0382   85.4273   86.1548   86.5672   
     서남권         -         -         -         -         -         -   
     동남권         -         -         -         -         -         -   

         2008년 01월 2008년 02월 2008년 03월 2008년 04월 2008년 05월 2008년 06월  \
NaN  NaN   85.8118   86.8334   89.3438   90.9571   92.5835   94.3929   
강북지역 NaN   84.3305   85.1741   87.7813   89.1495   91.1308   93.4676   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   
강남지역 서북권   87.3728   88.5882   90.9913   92.8711   94.1094   95.3491   
     서남권         -         -         -         -         -         -   
     동남권         -         -         -         -         -         -   

         2008년 07월 2008년 08월 2008년 09월 2008년 10월 2008년 11월 2008년 12월  \
NaN  NaN   95.5282   96.3193   97.0417   97.3262   96.8324     95.59   
강북지역 NaN   94.4015   94.7741   96.0298   96.4586   95.6234   94.7362   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   
강남지역 서북권   96.7004   97.9437   98.0898   98.2194   98.0925    96.469   
     서남권         -         -         -         -         -         -   
     동남권         -         -         -         -         -         -   

         2009년 01월 2009년 02월 2009년 03월 2009년 04월 2009년 05월 2009년 06월  \
NaN  NaN   95.0407   94.9244    94.809   94

In [99]:
yunrip_trade_price_index_df.index

MultiIndex([(   nan,   nan),
            ('강북지역',   nan),
            ('강북지역', '도심권'),
            ('강북지역', '동북권'),
            ('강북지역', '서북권'),
            ('강남지역', '서북권'),
            ('강남지역', '서남권'),
            ('강남지역', '동남권')],
           )

In [100]:
idx_list = []
for i in range(len(yunrip_trade_price_index_df.index)):
    if i == 0:
        idx_list.append('서울전체')
    elif i == 1:
        idx_list.append('강북전체')
    elif i == 5:
        idx_list.append('강남전체')
    else:
        idx_list.append(yunrip_trade_price_index_df.index[i][1])        

idx_list

['서울전체', '강북전체', '도심권', '동북권', '서북권', '강남전체', '서남권', '동남권']

In [101]:
yunrip_trade_price_index_df.index = idx_list
yunrip_trade_price_index_df

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,64.8385,65.1394,65.4442,65.8232,66.3765,66.971,67.2219,67.4802,68.4117,69.8208,72.1323,73.6332,75.2736,76.2046,76.9329,77.742,78.1654,78.7278,79.7804,80.489,81.8179,82.9367,83.7235,84.4694,85.8118,86.8334,89.3438,90.9571,92.5835,94.3929,95.5282,96.3193,97.0417,97.3262,96.8324,95.59,95.0407,94.9244,94.809,94.7801,94.8022,95.0276,95.668,96.1965,97.316,97.9983,98.323,98.4087,98.4999,98.7554,98.9797,99.0868,99.0276,98.9011,98.6945,98.3499,98.2756,98.2811,98.2634,98.2648,98.4198,98.5879,98.8252,99.05,99.3358,99.3779,99.3836,99.9004,99.9324,100.056,100.046,99.9396,99.839185,99.842606,99.623061,99.443781,99.268915,99.208559,99.028437,98.845848,98.553991,98.411479,98.198997,97.901373,97.421455,97.205527,97.022176,96.746070,96.669489,96.381454,96.075414,95.771403,95.696758,95.808761,95.761644,95.690992,95.719224,95.800828,95.788923,95.641919,95.555486,95.464836,95.389507,95.330367,95.346937,95.340251,95.406811,95.390326,95.420588,95.473676,95.609408,95.828185,96.064980,96.329496,96.526067,96.682529,96.877172,97.090931,97.333826,97.473593,97.527407,97.525737,97.544920,97.600592,97.665383,97.736621,97.814978,97.913882,98.015499,98.136544,98.186512,98.229065,98.246916,98.276070,98.363547,98.527646,98.751321,99.031750,99.294976,99.476385,99.567171,99.735174,100,100.235558,100.499953,100.891665,101.137834,101.321089,101.446669,101.628380,101.865527,102.203554,102.607602,102.910210,103.088440,103.164502,103.083730,103.010874,102.909812,102.834885,102.732019,102.634809,102.588160,102.601201,102.637529,102.786708,102.991850,103.363516,103.497693,103.618965,103.725386,103.736419,103.714942,103.772829,103.924512,104.163231,104.357338,104.514147,104.698007,104.901992
강북전체,64.265,64.3417,64.5677,64.8161,64.9957,65.3781,65.7757,65.9864,67.1408,68.6802,70.7498,72.2638,73.6366,74.4338,75.3608,76.1198,76.6601,77.1466,78.147,78.7437,79.7394,80.6139,81.453,82.4994,84.3305,85.1741,87.7813,89.1495,91.1308,93.4676,94.4015,94.7741,96.0298,96.4586,95.6234,94.7362,94.3421,94.3451,94.2515,94.3572,94.4724,94.6714,95.4413,95.941,96.8516,97.5147,97.8975,98.0014,98.169,98.4147,98.7012,98.7463,98.6966,98.5275,98.3757,98.0259,97.8769,97.9103,97.8531,97.8382,97.8714,97.9146,98.1936,98.5333,98.623

In [106]:
yunrip_trade_price_index_df.iloc[2,0] == '-'

True

In [115]:
도심권 = ['종로구', '중구', '용산구']
동북권 = ['성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구']
서북권 = ['은평구', '서대문구', '마포구']
서남권 = ['양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구']
동남권 = ['서초구', '강남구', '송파구', '강동구']

In [107]:
main_data_df['건물종류'].value_counts()

아파트      2632499
다세대연립    1375500
오피스텔      433257
Name: 건물종류, dtype: int64

In [108]:
yunrip_trade_df = main_data_df[(main_data_df['건물종류'] == '다세대연립')&(main_data_df['전월세매매구분'] == '매매')]
yunrip_trade_df.shape

(604199, 18)

In [110]:
yunrip_trade_df['구'] = yunrip_trade_df['지번주소'].apply(get_gu)
yunrip_trade_df['구'].value_counts()

은평구     67714
강서구     56315
양천구     35947
송파구     35438
강북구     34449
관악구     30219
도봉구     25846
구로구     25828
마포구     25684
강동구     25199
동작구     25112
광진구     25057
서대문구    23592
성북구     23065
중랑구     21172
서초구     16968
금천구     16471
용산구     16094
강남구     15729
노원구     13923
동대문구    13101
종로구     10888
영등포구     9207
성동구      7169
중구       3982
Name: 구, dtype: int64

In [116]:
for gu in tqdm(yunrip_trade_df['구'].unique()):
    if pd.isna(gu) == True:
        continue
    gu_df = yunrip_trade_df[yunrip_trade_df['구'] == gu]
    for col in yunrip_trade_price_index_df.columns:
        splitted = col.split(' ')
        year = int(splitted[0].replace('년',''))
        month = int(splitted[1].replace('월',''))
        date_df = gu_df[(gu_df['년'] == year)&(gu_df['월'] == month)]
        
        if gu in 도심권:
            area = '도심권'
        elif gu in 동북권:
            area = '동북권'
        elif gu in 서북권:
            area = '서북권'
        elif gu in 서남권:
            area = '서남권'
        elif gu in 동남권:
            area = '동남권'
        else:
            print(gu)
            raise ValueError('myerror')
        
        price_idx = yunrip_trade_price_index_df.loc[area, col]
        if price_idx == '-':
            if area in ['도심권', '동북권', '서북권']:
                price_idx = yunrip_trade_price_index_df.loc['강북전체', col]
            elif area in ['서남권', '동남권']:
                price_idx = yunrip_trade_price_index_df.loc['강남전체', col]
        
        date_df['주택가격지수'] = price_idx
        
        main_data_df.loc[date_df.index, '주택가격지수'] = date_df['주택가격지수']

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:10<00:00,  2.72s/it]


In [118]:
main_data_df['주택가격지수'].isna().sum()

2744075

In [119]:
main_data_df['지번주소'].isna().sum()

1556

In [120]:
filenames

['월간_매매가격지수_아파트.xlsx',
 '월간_매매가격지수_연립다세대.xlsx',
 '월간_매매가격지수_종합.xlsx',
 '월간_전세가격지수_아파트.xlsx',
 '월간_전세가격지수_연립다세대.xlsx',
 '월간_전세가격지수_종합.xlsx']

In [126]:
gross_trade_price_index_df = pd.read_excel(basedir + filenames[2], header=[10,11], index_col=[0,1])
print(gross_trade_price_index_df.shape)
gross_trade_price_index_df.head()

(26, 360)


지 역    2006년 01월       2006년 02월       2006년 03월       2006년 04월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    65.734  0.58    66.200  0.71    66.933  1.11    67.942  1.51   
   종로구    73.377  0.13    73.599  0.30    75.561  2.67    76.323  1.01   
   중구     73.069  0.42    73.329  0.36    73.979  0.89    74.503  0.71   
   용산구    58.761  0.57    59.237  0.81    59.852  1.04    63.758  6.53   
   성동구    77.805  0.36    78.417  0.79    79.184  0.98    79.889  0.89   

지 역    2006년 05월       2006년 06월       2006년 07월       2006년 08월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    68.901  1.41    69.423  0.76    69.706  0.41    69.944  0.34   
   종로구    76.834  0.67    76.921  0.11    76.866 -0.07    77.019  0.20   
   중구     74.731  0.31    75.281  0.74    75.523  0.32    75.499 -0.03   
   용산구    64.618  1.35    66.491  2.90    66.884  0.59    67.661  1.16   
   성동구    81.085  1.50    81.814  0.90    82.357  0.66    82.860  0.61   

지 역    2006년 09월       2006년 10월       2006년 11월       2006년 12월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    70.667  1.03    72.087  2.01    75.389  4.58    77.682  3.04   
   종로구    77.182  0.21    78.254  1.39    79.354  1.41    79.924  0.72   
   중구     75.896  0.53    77.200  1.72    78.480  1.66    80.364  2.40   
   용산구    72.885  7.72    74.121  1.70    76.126  2.71    79.240  4.09   
   성동구    82.972  0.14    84.308  1.61    87.114  3.33    89.389  2.61   

지 역    2007년 01월       2007년 02월       2007년 03월       2007년 04월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    79.305  2.09    79.988  0.86    80.509  0.65    80.984  0.59   
   종로구    80.279  0.44    80.577  0.37    80.806  0.28    80.923  0.14   
   중구     81.296  1.16    83.264  2.42    83.633  0.44    84.037  0.48   
   용산구    81.695  3.10    83.405  2.09    84.188  0.94    85.177  1.17   
   성동구    90.656  1.42    91.626  1.07    92.049  0.46    92.312  0.29   

지 역    2007년 05월       2007년 06월       2007년 07월       2007년 08월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    81.185  0.25    81.573  0.48    82.179  0.74    82.576  0.48   
   종로구    80.950  0.03    81.605  0.81    81.668  0.08    81.823  0.19   
   중구     84.312  0.33    84.701  0.46    85.059  0.42    85.119  0.07   
   용산구    86.981  2.12    89.264  2.62    89.799  0.60    90.008  0.23   
   성동구    92.363  0.05    92.536  0.19    92.719  0.20    92.856  0.15   

지 역    2007년 09월       2007년 10월       2007년 11월       2007년 12월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    83.359  0.95    84.035  0.81    84.815  0.93    85.305  0.58   
   종로구    81.883  0.07    82.036  0.19    82.063  0.03    82.454  0.48   
   중구     85.921  0.94    86.647  0.85    87.456  0.93    87.947  0.56   
   용산구    92.592  2.87    93.910  1.42    94.836  0.99    95.335  0.53   
   성동구    93.053  0.21    93.204  0.16    93.384  0.19    93.871  0.52   

지 역    2008년 01월       2008년 02월       2008년 03월       2008년 04월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    86.180  1.03    86.931  0.87    88.716  2.05    90.670  2.20   
   종로구    83.095  0.78    83.221  0.15    84.656  1.72    86.102  1.71   
   중구     88.458  0.58    89.139  0.77    89.624  0.54    90.331  0.79   
   용산구    96.729  1.46    98.002  1.32    99.124  1.14   100.029  0.91   
   성동구    94.453  0.62    94.575  0.13    95.344  0.81    95.981  0.67   

지 역    2008년 05월       2008년 06월       2008년 07월       2008년 08월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    92.098  1.58    93.702  1.74    94.558  0.91    94.973  0.44   
   종로구    86.699  0.69    86.835  0.16    87.153  0.37    87.290  0.16   
   중구     91.165  0.92    91.595  0.47    91.731  0.15    91.883  0.17 

In [127]:
cols_to_drop = [col for col in gross_trade_price_index_df.columns if '변동률' in col]
cols_to_drop

[('2006년 01월', '변동률'),
 ('2006년 02월', '변동률'),
 ('2006년 03월', '변동률'),
 ('2006년 04월', '변동률'),
 ('2006년 05월', '변동률'),
 ('2006년 06월', '변동률'),
 ('2006년 07월', '변동률'),
 ('2006년 08월', '변동률'),
 ('2006년 09월', '변동률'),
 ('2006년 10월', '변동률'),
 ('2006년 11월', '변동률'),
 ('2006년 12월', '변동률'),
 ('2007년 01월', '변동률'),
 ('2007년 02월', '변동률'),
 ('2007년 03월', '변동률'),
 ('2007년 04월', '변동률'),
 ('2007년 05월', '변동률'),
 ('2007년 06월', '변동률'),
 ('2007년 07월', '변동률'),
 ('2007년 08월', '변동률'),
 ('2007년 09월', '변동률'),
 ('2007년 10월', '변동률'),
 ('2007년 11월', '변동률'),
 ('2007년 12월', '변동률'),
 ('2008년 01월', '변동률'),
 ('2008년 02월', '변동률'),
 ('2008년 03월', '변동률'),
 ('2008년 04월', '변동률'),
 ('2008년 05월', '변동률'),
 ('2008년 06월', '변동률'),
 ('2008년 07월', '변동률'),
 ('2008년 08월', '변동률'),
 ('2008년 09월', '변동률'),
 ('2008년 10월', '변동률'),
 ('2008년 11월', '변동률'),
 ('2008년 12월', '변동률'),
 ('2009년 01월', '변동률'),
 ('2009년 02월', '변동률'),
 ('2009년 03월', '변동률'),
 ('2009년 04월', '변동률'),
 ('2009년 05월', '변동률'),
 ('2009년 06월', '변동률'),
 ('2009년 07월', '변동률'),
 ('2009년 08

In [128]:
gross_trade_price_index_df.drop(columns=cols_to_drop, inplace=True)
print(gross_trade_price_index_df.shape)

(26, 180)


In [129]:
gross_trade_price_index_df.columns = [col[0] for col in gross_trade_price_index_df.columns]
gross_trade_price_index_df.head()

2006년 01월  2006년 02월  2006년 03월  2006년 04월  2006년 05월  2006년 06월  \
서울 NaN     65.734     66.200     66.933     67.942     68.901     69.423   
   종로구     73.377     73.599     75.561     76.323     76.834     76.921   
   중구      73.069     73.329     73.979     74.503     74.731     75.281   
   용산구     58.761     59.237     59.852     63.758     64.618     66.491   
   성동구     77.805     78.417     79.184     79.889     81.085     81.814   

        2006년 07월  2006년 08월  2006년 09월  2006년 10월  2006년 11월  2006년 12월  \
서울 NaN     69.706     69.944     70.667     72.087     75.389     77.682   
   종로구     76.866     77.019     77.182     78.254     79.354     79.924   
   중구      75.523     75.499     75.896     77.200     78.480     80.364   
   용산구     66.884     67.661     72.885     74.121     76.126     79.240   
   성동구     82.357     82.860     82.972     84.308     87.114     89.389   

        2007년 01월  2007년 02월  2007년 03월  2007년 04월  2007년 05월  2007년 06월  \
서울 NaN     79.305     79.988     80.509     80.984     81.185     81.573   
   종로구     80.279     80.577     80.806     80.923     80.950     81.605   
   중구      81.296     83.264     83.633     84.037     84.312     84.701   
   용산구     81.695     83.405     84.188     85.177     86.981     89.264   
   성동구     90.656     91.626     92.049     92.312     92.363     92.536   

        2007년 07월  2007년 08월  2007년 09월  2007년 10월  2007년 11월  2007년 12월  \
서울 NaN     82.179     82.576     83.359     84.035     84.815     85.305   
   종로구     81.668     81.823     81.883     82.036     82.063     82.454   
   중구      85.059     85.119     85.921     86.647     87.456     87.947   
   용산구     89.799     90.008     92.592     93.910     94.836     95.335   
   성동구     92.719     92.856     93.053     93.204     93.384     93.871   

        2008년 01월  2008년 02월  2008년 03월  2008년 04월  2008년 05월  2008년 06월  \
서울 NaN     86.180     86.931     88.716     90.670     92.098     93.702   
   종로구     83.095     83.221     84.656     86.102     86.699     86.835   
   중구      88.458     89.139     89.624     90.331     91.165     91.595   
   용산구     96.729     98.002     99.124    100.029    100.538    102.300   
   성동구     94.453     94.575     95.344     95.981     96.535     96.737   

        2008년 07월  2008년 08월  2008년 09월  2008년 10월  2008년 11월  2008년 12월  \
서울 NaN     94.558     94.973     95.443     95.428     94.773     93.460   
   종로구     87.153     87.290     87.701     88.867     89.090     88.929   
   중구      91.731     91.883     92.149     92.416     92.185     91.974   
   용산구    103.013    103.150    103.286    103.381    101.402    100.153   
   성동구     97.078     97.115     97.303     97.268     96.981     95.830   

        2009년 01월  2009년 02월  2009년 03월  2009년 04월  2009년 05월  2009년 06월  \
서울 NaN     92.810     92.702     92.563     92.789     92.905     93.245   
   종로구     88.964     88.919     88.978     89.307     89.328     89.690   
   중구      91.785     92.072     92.304     92.381     92.676     92.959   
   용산구     99.669     99.458     98.988    100.910    100.976    101.091   
   성동구     95.603     95.448     95.413     95.553     95.617     95.711   

        2009년 07월  2009년 08월  2009년 09월  2009년 10월  2009년 11월  2009년 12월  \
서울 NaN     93.914     94.423     95.480     95.938     96.046     96.086   
   종로구     90.177     90.736     91.749     92.317     92.292     92.312   
   중구      93.445     93.543     94.015     94.221     94.481     94.514   
   용산구    101.394    101.939    102.359    102.516    102.648    102.727   
   성동구     95.716     95.744     95.932     96.032     96.043     95.992   

        2010년 01월  2010년 02월  2010년 03월  2010년 04월  2010년 05월  2010년 06월  \
서울 NaN     96.211     96.431     96.503     96.486     96.240     96.006   
   종로구     92.668     92.703     92.874     92.857     92.843     92.767   
   중구      94.582     94.778     95.217     95.475     95.403     95.342   
   용산구    102.737    102.774    102.808    102.8

In [130]:
gross_trade_price_index_df = gross_trade_price_index_df.loc['서울']
gross_trade_price_index_df.head()

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
NaN,65.734,66.200,66.933,67.942,68.901,69.423,69.706,69.944,70.667,72.087,75.389,77.682,79.305,79.988,80.509,80.984,81.185,81.573,82.179,82.576,83.359,84.035,84.815,85.305,86.180,86.931,88.716,90.670,92.098,93.702,94.558,94.973,95.443,95.428,94.773,93.460,92.810,92.702,92.563,92.789,92.905,93.245,93.914,94.423,95.480,95.938,96.046,96.086,96.211,96.431,96.503,96.486,96.240,96.006,95.681,95.319,95.142,95.032,94.974,95.006,95.164,95.387,95.601,95.733,95.855,95.837,95.772,95.886,95.865,95.855,95.776,95.661,95.505,95.368,95.053,94.593,94.223,93.756,93.327,92.755,92.296,91.913,91.573,91.121,90.656,90.379,90.306,90.344,90.478,90.265,89.878,89.506,89.510,89.808,89.864,89.837,90.042,90.252,90.454,90.444,90.380,90.312,90.273,90.316,90.538,90.764,90.852,90.852,90.930,91.113,91.439,91.909,92.310,92.772,93.122,93.467,93.886,94.322,94.755,95.030,95.075,95.077,95.086,95.159,95.297,95.510,95.729,95.973,96.224,96.638,96.972,97.064,97.088,97.140,97.271,97.496,97.838,98.486,98.888,99.335,99.407,99.638,100,100.593,101.457,102.411,102.972,103.293,103.514,103.754,104.085,104.742,106.048,106.588,106.807,106.848,106.630,106.432,106.199,106.011,105.899,105.859,105.932,106.079,106.258,106.730,107.258,108.183,108.547,108.708,108.852,108.831,108.737,108.878,109.655,110.119,110.419,110.597,110.782,111.072
종로구,73.377,73.599,75.561,76.323,76.834,76.921,76.866,77.019,77.182,78.254,79.354,79.924,80.279,80.577,80.806,80.923,80.950,81.605,81.668,81.823,81.883,82.036,82.063,82.454,83.095,83.221,84.656,86.102,86.699,86.835,87.153,87.290,87.701,88.867,89.090,88.929,88.964,88.919,88.978,89.307,89.328,89.690,90.177,90.736,91.749,92.317,92.292,92.312,92.668,92.703,92.874,92.857,92.843,92.767,92.694,92.619,92.717,92.778,92.786,92.815,93.103,93.154,93.299,93.389,94.091,94.180,94.154,94.315,94.355,94.357,94.783,94.859,94.859,94.859,94.859,94.846,94.809,94.806,94.747,94.561,94.545,94.406,94.391,94.265,94.265,94.265,94.244,94.352,94.342,94.069,93.925,93.756,93.684,93.843,93.879,93.895,93.905,94.002,94.006,94.018,94.012,93.990,93.996,93.993,94.113,94.129,94.125,94.120,94.176,94.202,94.332,94.542,94.733,95.163,95.437,95.533,95.648,95.816,96.086,96.135,96.203,96.210,96.215,96.243,96.311,96.472,96.636,96.829,9

In [131]:
gross_trade_price_index_df.index

Index([   nan,  '종로구',   '중구',  '용산구',  '성동구',  '광진구', '동대문구',  '중랑구',  '성북구',
        '강북구',  '도봉구',  '노원구',  '은평구', '서대문구',  '마포구',  '양천구',  '강서구',  '구로구',
        '금천구', '영등포구',  '동작구',  '관악구',  '서초구',  '강남구',  '송파구',  '강동구'],
      dtype='object')

In [132]:
idx_list = []
for i in range(len(gross_trade_price_index_df.index)):
    if i == 0:
        idx_list.append('서울전체')
    else:
        idx_list.append(gross_trade_price_index_df.index[i])
idx_list

['서울전체',
 '종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [133]:
gross_trade_price_index_df.index = idx_list
gross_trade_price_index_df

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,65.734,66.200,66.933,67.942,68.901,69.423,69.706,69.944,70.667,72.087,75.389,77.682,79.305,79.988,80.509,80.984,81.185,81.573,82.179,82.576,83.359,84.035,84.815,85.305,86.180,86.931,88.716,90.670,92.098,93.702,94.558,94.973,95.443,95.428,94.773,93.460,92.810,92.702,92.563,92.789,92.905,93.245,93.914,94.423,95.480,95.938,96.046,96.086,96.211,96.431,96.503,96.486,96.240,96.006,95.681,95.319,95.142,95.032,94.974,95.006,95.164,95.387,95.601,95.733,95.855,95.837,95.772,95.886,95.865,95.855,95.776,95.661,95.505,95.368,95.053,94.593,94.223,93.756,93.327,92.755,92.296,91.913,91.573,91.121,90.656,90.379,90.306,90.344,90.478,90.265,89.878,89.506,89.510,89.808,89.864,89.837,90.042,90.252,90.454,90.444,90.380,90.312,90.273,90.316,90.538,90.764,90.852,90.852,90.930,91.113,91.439,91.909,92.310,92.772,93.122,93.467,93.886,94.322,94.755,95.030,95.075,95.077,95.086,95.159,95.297,95.510,95.729,95.973,96.224,96.638,96.972,97.064,97.088,97.140,97.271,97.496,97.838,98.486,98.888,99.335,99.407,99.638,100,100.593,101.457,102.411,102.972,103.293,103.514,103.754,104.085,104.742,106.048,106.588,106.807,106.848,106.630,106.432,106.199,106.011,105.899,105.859,105.932,106.079,106.258,106.730,107.258,108.183,108.547,108.708,108.852,108.831,108.737,108.878,109.655,110.119,110.419,110.597,110.782,111.072
종로구,73.377,73.599,75.561,76.323,76.834,76.921,76.866,77.019,77.182,78.254,79.354,79.924,80.279,80.577,80.806,80.923,80.950,81.605,81.668,81.823,81.883,82.036,82.063,82.454,83.095,83.221,84.656,86.102,86.699,86.835,87.153,87.290,87.701,88.867,89.090,88.929,88.964,88.919,88.978,89.307,89.328,89.690,90.177,90.736,91.749,92.317,92.292,92.312,92.668,92.703,92.874,92.857,92.843,92.767,92.694,92.619,92.717,92.778,92.786,92.815,93.103,93.154,93.299,93.389,94.091,94.180,94.154,94.315,94.355,94.357,94.783,94.859,94.859,94.859,94.859,94.846,94.809,94.806,94.747,94.561,94.545,94.406,94.391,94.265,94.265,94.265,94.244,94.352,94.342,94.069,93.925,93.756,93.684,93.843,93.879,93.895,93.905,94.002,94.006,94.018,94.012,93.990,93.996,93.993,94.113,94.129,94.125,94.120,94.176,94.202,94.332,94.542,94.733,95.163,95.437,95.533,95.648,95.816,96.086,96.135,96.203,96.210,96.215,96.243,96.311,96.472,96.636,96.829,

In [134]:
main_data_df['건물종류'].value_counts()

아파트      2632499
다세대연립    1375500
오피스텔      433257
Name: 건물종류, dtype: int64

In [135]:
officetel_trade_df = main_data_df[(main_data_df['건물종류'] == '오피스텔')&(main_data_df['전월세매매구분'] == '매매')]
officetel_trade_df.shape

(166261, 18)

In [136]:
officetel_trade_df['구'] = officetel_trade_df['지번주소'].apply(get_gu)
officetel_trade_df['구'].value_counts()

영등포구    19178
강남구     17015
마포구     13016
강서구     11914
서초구     11636
구로구     11190
동대문구     7507
관악구      7434
종로구      6988
송파구      6855
양천구      6147
용산구      5488
강동구      5144
성동구      4788
서대문구     4568
노원구      4009
중구       3976
광진구      3894
은평구      3530
도봉구      3471
금천구      2332
강북구      1946
중랑구      1688
동작구      1517
성북구       988
Name: 구, dtype: int64

In [138]:
for gu in tqdm(officetel_trade_df['구'].unique()):
    if pd.isna(gu) == True:
        continue
    gu_df = officetel_trade_df[officetel_trade_df['구'] == gu]
    for col in gross_trade_price_index_df.columns:
        splitted = col.split(' ')
        year = int(splitted[0].replace('년',''))
        month = int(splitted[1].replace('월',''))
        date_df = gu_df[(gu_df['년'] == year)&(gu_df['월'] == month)]
        date_df['주택가격지수'] = gross_trade_price_index_df.loc[gu, col]
        main_data_df.loc[date_df.index, '주택가격지수'] = date_df['주택가격지수']

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:10<00:00,  2.72s/it]


In [139]:
filenames

['월간_매매가격지수_아파트.xlsx',
 '월간_매매가격지수_연립다세대.xlsx',
 '월간_매매가격지수_종합.xlsx',
 '월간_전세가격지수_아파트.xlsx',
 '월간_전세가격지수_연립다세대.xlsx',
 '월간_전세가격지수_종합.xlsx']

In [140]:
apart_lease_price_index_df = pd.read_excel(basedir + filenames[3], header=[10,11], index_col=[0,1])
print(apart_lease_price_index_df.shape)
apart_lease_price_index_df.head()

(26, 360)


지 역    2006년 01월       2006년 02월       2006년 03월       2006년 04월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    50.844  0.86    51.344  0.98    52.132  1.53    52.750  1.18   
   종로구    60.064  0.17    60.652  0.98    61.568  1.51    61.830  0.43   
   중구     54.221 -0.29    54.467  0.45    54.535  0.12    54.840  0.56   
   용산구    56.288  0.36    56.757  0.83    57.268  0.90    58.227  1.67   
   성동구    45.516  0.27    45.605  0.20    46.639  2.27    47.183  1.17   

지 역    2006년 05월       2006년 06월       2006년 07월       2006년 08월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    53.071  0.61    53.182  0.21    53.297  0.22    53.502  0.38   
   종로구    61.830  0.00    62.196  0.59    61.964 -0.37    61.996  0.05   
   중구     55.100  0.47    56.235  2.06    56.406  0.30    56.520  0.20   
   용산구    59.132  1.55    59.401  0.46    59.468  0.11    59.496  0.05   
   성동구    47.517  0.71    47.754  0.50    47.893  0.29    48.132  0.50   

지 역    2006년 09월       2006년 10월       2006년 11월       2006년 12월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    54.157  1.23    55.066  1.68    55.996  1.69    56.314  0.57   
   종로구    62.386  0.63    62.692  0.49    64.310  2.58    64.503  0.30   
   중구     56.941  0.74    59.136  3.86    60.433  2.19    61.404  1.61   
   용산구    59.712  0.36    60.114  0.67    61.068  1.59    61.551  0.79   
   성동구    48.541  0.85    49.587  2.16    50.039  0.91    50.551  1.02   

지 역    2007년 01월       2007년 02월       2007년 03월       2007년 04월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    56.693  0.67    56.976  0.50    57.358  0.67    57.697  0.59   
   종로구    65.035  0.82    65.531  0.76    65.268 -0.40    65.296  0.04   
   중구     61.684  0.46    61.818  0.22    61.957  0.22    61.975  0.03   
   용산구    61.806  0.41    61.703 -0.17    61.844  0.23    62.054  0.34   
   성동구    50.933  0.76    51.669  1.45    52.184  1.00    52.277  0.18   

지 역    2007년 05월       2007년 06월       2007년 07월       2007년 08월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    57.685 -0.02    57.690  0.01    57.732  0.07    57.851  0.21   
   종로구    65.378  0.13    65.500  0.19    65.871  0.57    65.951  0.12   
   중구     62.503  0.85    62.506  0.01    62.375 -0.21    62.375  0.00   
   용산구    62.032 -0.04    62.068  0.06    62.205  0.22    62.211  0.01   
   성동구    52.196 -0.16    52.147 -0.09    52.268  0.23    52.268  0.00   

지 역    2007년 09월       2007년 10월       2007년 11월       2007년 12월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    57.934  0.14    57.993  0.10    57.978 -0.03    57.939 -0.07   
   종로구    66.527  0.87    66.649  0.18    66.649  0.00    66.664  0.02   
   중구     62.460  0.14    62.389 -0.11    62.245 -0.23    62.136 -0.17   
   용산구    62.293  0.13    62.199 -0.15    62.199  0.00    62.498  0.48   
   성동구    52.185 -0.16    52.199  0.03    52.405  0.39    52.501  0.18   

지 역    2008년 01월       2008년 02월       2008년 03월       2008년 04월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    58.065  0.22    58.233  0.29    58.656  0.73    59.125  0.80   
   종로구    66.593 -0.11    67.099  0.76    67.651  0.82    68.588  1.38   
   중구     62.062 -0.12    62.287  0.36    62.339  0.08    62.526  0.30   
   용산구    62.538  0.06    62.568  0.05    62.680  0.18    63.007  0.52   
   성동구    52.478 -0.04    52.519  0.08    53.273  1.44    53.415  0.27   

지 역    2008년 05월       2008년 06월       2008년 07월       2008년 08월        \
              지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN    59.317  0.32    59.459  0.24    59.614  0.26    59.652  0.06   
   종로구    68.624  0.05    68.675  0.07    68.940  0.39    68.931 -0.01   
   중구     62.994  0.75    63.111  0.19    63.079 -0.05    62.950 -0.20 

In [141]:
cols_to_drop = [col for col in apart_lease_price_index_df.columns if '변동률' in col]
cols_to_drop

[('2006년 01월', '변동률'),
 ('2006년 02월', '변동률'),
 ('2006년 03월', '변동률'),
 ('2006년 04월', '변동률'),
 ('2006년 05월', '변동률'),
 ('2006년 06월', '변동률'),
 ('2006년 07월', '변동률'),
 ('2006년 08월', '변동률'),
 ('2006년 09월', '변동률'),
 ('2006년 10월', '변동률'),
 ('2006년 11월', '변동률'),
 ('2006년 12월', '변동률'),
 ('2007년 01월', '변동률'),
 ('2007년 02월', '변동률'),
 ('2007년 03월', '변동률'),
 ('2007년 04월', '변동률'),
 ('2007년 05월', '변동률'),
 ('2007년 06월', '변동률'),
 ('2007년 07월', '변동률'),
 ('2007년 08월', '변동률'),
 ('2007년 09월', '변동률'),
 ('2007년 10월', '변동률'),
 ('2007년 11월', '변동률'),
 ('2007년 12월', '변동률'),
 ('2008년 01월', '변동률'),
 ('2008년 02월', '변동률'),
 ('2008년 03월', '변동률'),
 ('2008년 04월', '변동률'),
 ('2008년 05월', '변동률'),
 ('2008년 06월', '변동률'),
 ('2008년 07월', '변동률'),
 ('2008년 08월', '변동률'),
 ('2008년 09월', '변동률'),
 ('2008년 10월', '변동률'),
 ('2008년 11월', '변동률'),
 ('2008년 12월', '변동률'),
 ('2009년 01월', '변동률'),
 ('2009년 02월', '변동률'),
 ('2009년 03월', '변동률'),
 ('2009년 04월', '변동률'),
 ('2009년 05월', '변동률'),
 ('2009년 06월', '변동률'),
 ('2009년 07월', '변동률'),
 ('2009년 08

In [142]:
apart_lease_price_index_df.drop(columns=cols_to_drop)
apart_lease_price_index_df = apart_lease_price_index_df.loc['서울']
idx_list = []
for i in range(len(apart_lease_price_index_df.index)):
    if i == 0:
        idx_list.append('서울전체')
    else:
        idx_list.append(apart_lease_price_index_df.index[i])
idx_list

['서울전체',
 '종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [143]:
apart_lease_price_index_df.index = idx_list
apart_lease_price_index_df

지 역  2006년 01월       2006년 02월       2006년 03월       2006년 04월        \
            지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울전체    50.844  0.86    51.344  0.98    52.132  1.53    52.750  1.18   
종로구     60.064  0.17    60.652  0.98    61.568  1.51    61.830  0.43   
중구      54.221 -0.29    54.467  0.45    54.535  0.12    54.840  0.56   
용산구     56.288  0.36    56.757  0.83    57.268  0.90    58.227  1.67   
성동구     45.516  0.27    45.605  0.20    46.639  2.27    47.183  1.17   
광진구     49.990  1.62    50.843  1.71    51.432  1.16    52.027  1.16   
동대문구    48.842  0.33    49.210  0.75    49.500  0.59    49.700  0.41   
중랑구     51.841  0.22    51.945  0.20    52.585  1.23    53.262  1.29   
성북구     49.731  0.39    50.244  1.03    51.208  1.92    51.890  1.33   
강북구     48.509  0.64    49.077  1.17    49.598  1.06    49.900  0.61   
도봉구     51.264  0.91    51.602  0.66    52.457  1.66    52.888  0.82   
노원구     47.035  1.40    47.563  1.12    48.363  1.68    48.920  1.15   
은평구     58.155  0.13    58.413  0.44    58.430  0.03    58.783  0.60   
서대문구    49.933  0.48    50.120  0.37    50.462  0.68    51.210  1.48   
마포구     50.872  0.53    51.380  1.00    51.846  0.91    52.290  0.86   
양천구     51.351  2.48    52.292  1.83    53.975  3.22    55.137  2.15   
강서구     45.709  1.57    46.275  1.24    47.841  3.38    48.635  1.66   
구로구     45.365  0.03    45.977  1.35    46.740  1.66    47.345  1.30   
금천구     48.653  0.70    49.064  0.84    49.975  1.86    50.550  1.15   
영등포구    46.295  0.37    46.532  0.51    47.261  1.57    48.180  1.94   
동작구     50.211  0.85    51.144  1.86    52.201  2.07    52.856  1.26   
관악구     49.896  0.21    50.962  2.14    51.818  1.68    52.552  1.42   
서초구     59.009  0.64    59.441  0.73    60.074  1.06    60.499  0.71   
강남구     57.220  0.84    57.715  0.87    58.038  0.56    58.813  1.34   
송파구     56.177  1.23    56.672  0.88    57.230  0.98    57.588  0.63   
강동구     52.683  0.66    52.944  0.50    53.477  1.01    53.877  0.75   

지 역  2006년 05월       2006년 06월       2006년 07월       2006년 08월        \
            지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울전체    53.071  0.61    53.182  0.21    53.297  0.22    53.502  0.38   
종로구     61.830  0.00    62.196  0.59    61.964 -0.37    61.996  0.05   
중구      55.100  0.47    56.235  2.06    56.406  0.30    56.520  0.20   
용산구     59.132  1.55    59.401  0.46    59.468  0.11    59.496  0.05   
성동구     47.517  0.71    47.754  0.50    47.893  0.29    48.132  0.50   
광진구     52.491  0.89    52.576  0.16    52.752  0.33    52.980  0.43   
동대문구    50.342  1.29    50.408  0.13    50.474  0.13    51.469  1.97   
중랑구     53.685  0.79    53.758  0.13    53.785  0.05    54.528  1.38   
성북구     52.051  0.31    52.156  0.20    52.213  0.11    52.638  0.82   
강북구     49.956  0.11    50.006  0.10    49.950 -0.11    50.239  0.58   
도봉구     53.006  0.22    53.244  0.45    53.552  0.58    53.732  0.33   
노원구     49.432  1.05    49.473  0.08    49.633  0.32    50.020  0.78   
은평구     59.046  0.45    59.234  0.32    59.398  0.28    60.129  1.23   
서대문구    51.613  0.79    51.719  0.20    51.900  0.35    52.023  0.24   
마포구     52.773  0.92    53.460  1.30    53.741  0.53    54.038  0.55   
양천구     55.465  0.59    55.460 -0.01    55.745  0.51    55.574 -0.31   
강서구     48.905  0.56    49.336  0.88    49.546  0.43    49.698  0.31   
구로구     48.152  1.70    48.074 -0.16    48.152  0.16    48.306  0.32   
금천구     50.846  0.59    50.986  0.28    51.350  0.71    51.865  1.00   
영등포구    48.952  1.60    49.220  0.55    49.246  0.05    49.532  0.58   
동작구     53.086  0.43    53.388  0.57    53.662  0.51    53.303 -0.67   
관악구     53.023  0.90    53.487  0.87    53.401 -0.16    53.470  0.13   
서초구     59.573 -1.53    59.309 -0.44    59.246 -0.11    59.118 -0.22   
강남구     59.256  0.75    59.119 -0.23    59.274  0.26    59.310  0.06   
송파구     57.668  0.14    57.483 -0.32    57.383 -0.17    57.517  0.23   
강동구     53.958  0.15    53.930 -0.05   

In [144]:
apart_lease_price_index_df.drop(columns=cols_to_drop, inplace=True)
apart_lease_price_index_df.columns = [col[0] for col in apart_lease_price_index_df.columns]
apart_lease_price_index_df

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,50.844,51.344,52.132,52.750,53.071,53.182,53.297,53.502,54.157,55.066,55.996,56.314,56.693,56.976,57.358,57.697,57.685,57.690,57.732,57.851,57.934,57.993,57.978,57.939,58.065,58.233,58.656,59.125,59.317,59.459,59.614,59.652,59.754,59.602,58.951,57.285,56.288,56.364,56.718,57.036,57.360,57.665,58.174,58.716,60.329,60.962,61.467,61.738,62.065,62.660,63.092,63.401,63.592,63.698,63.767,63.898,64.324,65.047,65.735,66.239,66.959,68.306,69.553,70.200,70.124,70.439,71.209,72.419,73.966,74.731,74.955,74.824,74.731,74.741,74.658,74.346,74.007,73.726,73.321,73.124,73.568,74.179,74.675,74.842,75.127,75.573,76.091,76.364,76.510,76.616,77.133,77.962,78.970,80.213,81.016,81.594,82.576,83.308,83.943,84.072,84.060,84.079,84.241,84.425,84.804,85.264,85.579,85.904,86.245,86.826,87.606,88.699,89.468,90.332,91.029,91.658,92.590,93.456,94.451,95.173,95.498,95.720,95.875,96.033,96.236,96.490,96.769,96.956,97.120,97.379,97.705,97.857,97.924,97.970,98.124,98.322,98.586,98.982,99.232,99.501,99.626,99.764,100,100.210,100.465,100.665,100.501,100.147,99.737,99.498,99.547,99.826,100.194,100.425,100.398,100.181,99.665,98.973,98.351,98.037,97.896,97.836,97.854,97.996,98.160,98.511,98.913,99.490,100.203,100.447,100.611,100.721,100.784,101.023,101.473,102.133,102.745,103.235,104.045,105.041
종로구,60.064,60.652,61.568,61.830,61.830,62.196,61.964,61.996,62.386,62.692,64.310,64.503,65.035,65.531,65.268,65.296,65.378,65.500,65.871,65.951,66.527,66.649,66.649,66.664,66.593,67.099,67.651,68.588,68.624,68.675,68.940,68.931,69.284,69.347,68.305,67.553,66.973,66.518,66.646,66.684,66.699,66.874,66.662,67.221,69.339,69.921,70.243,70.302,70.349,70.433,70.753,70.766,70.893,70.954,70.952,70.975,71.415,71.696,71.911,73.028,73.381,74.054,75.766,76.480,76.770,76.940,77.102,77.880,79.375,80.547,80.984,80.977,80.977,80.977,80.977,81.181,81.181,81.181,81.228,81.653,81.888,81.993,82.066,82.085,82.159,82.232,82.879,83.351,83.603,83.859,84.324,84.661,85.188,85.753,86.006,86.130,86.668,87.183,87.382,87.516,87.536,87.528,87.524,87.680,88.226,88.488,88.583,88.651,88.804,88.947,89.637,90.204,90.490,90.826,91.141,91.343,91.830,92.308,92.874,93.080,93.203,93.295,93.380,93.521,93.937,94.187,94.696,95.122,95.343,95.794,96

In [145]:
apart_lease_df = main_data_df[(main_data_df['건물종류'] == '아파트')&(main_data_df['전월세매매구분'] == '전세')]
apart_lease_df.shape

(1112738, 18)

In [147]:
apart_lease_df['구'] = apart_lease_df['지번주소'].apply(get_gu)

In [148]:
for gu in tqdm(apart_lease_df['구'].unique()):
    if pd.isna(gu) == True:
        continue
    gu_df = apart_lease_df[apart_lease_df['구'] == gu]
    for col in apart_lease_price_index_df.columns:
        splitted = col.split(' ')
        year = int(splitted[0].replace('년',''))
        month = int(splitted[1].replace('월',''))
        date_df = gu_df[(gu_df['년'] == year)&(gu_df['월'] == month)]
        date_df['주택가격지수'] = apart_lease_price_index_df.loc[gu, col]
        main_data_df.loc[date_df.index, '주택가격지수'] = date_df['주택가격지수']

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:11<00:00,  2.74s/it]


In [149]:
main_data_df['주택가격지수'].isna().sum()

1466125

In [150]:
filenames

['월간_매매가격지수_아파트.xlsx',
 '월간_매매가격지수_연립다세대.xlsx',
 '월간_매매가격지수_종합.xlsx',
 '월간_전세가격지수_아파트.xlsx',
 '월간_전세가격지수_연립다세대.xlsx',
 '월간_전세가격지수_종합.xlsx']

In [152]:
yunrip_lease_price_index_df = pd.read_excel(basedir + filenames[4], header=[10,11], index_col=[0,1,2])
print(yunrip_lease_price_index_df.shape)
yunrip_lease_price_index_df.head()

(8, 362)


지 역         2005년 12월     2006년 01월       2006년 02월       2006년 03월        \
                   지수 변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    57.028   -    57.204  0.31    57.452  0.43    57.989  0.93   
   강북지역 NaN    56.955   -     57.09  0.24    57.204   0.2    57.736  0.93   
        도심권         -   -         -     -         -     -         -     -   
        동북권         -   -         -     -         -     -         -     -   
        서북권         -   -         -     -         -     -         -     -   

지 역         2006년 04월       2006년 05월       2006년 06월       2006년 07월        \
                   지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    58.382  0.68    58.568  0.32    58.806  0.41    58.897  0.15   
   강북지역 NaN    58.081   0.6    58.156  0.13    58.312  0.27    58.375  0.11   
        도심권         -     -         -     -         -     -         -     -   
        동북권         -     -         -     -         -     -         -     -   
        서북권         -     -         -     -         -     -         -     -   

지 역         2006년 08월       2006년 09월       2006년 10월       2006년 11월        \
                   지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    59.194  0.51    59.883  1.16    60.731  1.42    61.629  1.48   
   강북지역 NaN    58.814  0.75    59.494  1.16    60.813  2.22    61.656  1.39   
        도심권         -     -         -     -         -     -         -     -   
        동북권         -     -         -     -         -     -         -     -   
        서북권         -     -         -     -         -     -         -     -   

지 역         2006년 12월       2007년 01월       2007년 02월       2007년 03월        \
                   지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    62.125  0.81    62.511  0.62     63.07  0.89    63.611  0.86   
   강북지역 NaN    62.085   0.7    62.492  0.65    62.895  0.65     63.49  0.94   
        도심권         -     -         -     -         -     -         -     -   
        동북권         -     -         -     -         -     -         -     -   
        서북권         -     -         -     -         -     -         -     -   

지 역         2007년 04월       2007년 05월       2007년 06월       2007년 07월        \
                   지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    64.127  0.81    64.495  0.57    64.742  0.38    65.289  0.84   
   강북지역 NaN    64.089  0.94     64.42  0.52    64.554  0.21    65.162  0.94   
        도심권         -     -         -     -         -     -         -     -   
        동북권         -     -         -     -         -     -         -     -   
        서북권         -     -         -     -         -     -         -     -   

지 역         2007년 08월       2007년 09월       2007년 10월       2007년 11월        \
                   지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    65.583  0.45    66.628  1.59    67.292     1    67.727  0.65   
   강북지역 NaN    65.494  0.51    66.473  1.49    67.046  0.86    67.743  1.04   
        도심권         -     -         -     -         -     -         -     -   
        동북권         -     -         -     -         -     -         -     -   
        서북권         -     -         -     -         -     -         -     -   

지 역         2007년 12월       2008년 01월       2008년 02월       2008년 03월        \
                   지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN  NaN    68.052  0.48      68.6  0.81    69.265  0.97    70.375   1.6   
   강북지역 NaN    68.209  0.69    69.016  1.18    69.636   0.9    71.052  2.03   
        도심권         -     -         -     -         -     -         -     -   
        동북권         -     -         -     -         -     -         -     -   
        서북권         -     -         -     -         -     -         -     -   

지 역         2008년 04월       2008년 05월       2008년 06월       2008년 07월        \
                   지수   변동률        지수   변동률        지수   변

In [153]:
cols_to_drop = [col for col in yunrip_lease_price_index_df.columns if '변동률' in col]
cols_to_drop[:5]

[('2005년 12월', '변동률'),
 ('2006년 01월', '변동률'),
 ('2006년 02월', '변동률'),
 ('2006년 03월', '변동률'),
 ('2006년 04월', '변동률')]

In [154]:
yunrip_lease_price_index_df.drop(columns=cols_to_drop, inplace=True)

In [158]:
yunrip_lease_price_index_df.columns = [col[0] for col in yunrip_lease_price_index_df.columns]
yunrip_lease_price_index_df.head()

2005년 12월 2006년 01월 2006년 02월 2006년 03월 2006년 04월 2006년 05월  \
서울 NaN  NaN    57.028    57.204    57.452    57.989    58.382    58.568   
   강북지역 NaN    56.955     57.09    57.204    57.736    58.081    58.156   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2006년 06월 2006년 07월 2006년 08월 2006년 09월 2006년 10월 2006년 11월  \
서울 NaN  NaN    58.806    58.897    59.194    59.883    60.731    61.629   
   강북지역 NaN    58.312    58.375    58.814    59.494    60.813    61.656   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2006년 12월 2007년 01월 2007년 02월 2007년 03월 2007년 04월 2007년 05월  \
서울 NaN  NaN    62.125    62.511     63.07    63.611    64.127    64.495   
   강북지역 NaN    62.085    62.492    62.895     63.49    64.089     64.42   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2007년 06월 2007년 07월 2007년 08월 2007년 09월 2007년 10월 2007년 11월  \
서울 NaN  NaN    64.742    65.289    65.583    66.628    67.292    67.727   
   강북지역 NaN    64.554    65.162    65.494    66.473    67.046    67.743   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2007년 12월 2008년 01월 2008년 02월 2008년 03월 2008년 04월 2008년 05월  \
서울 NaN  NaN    68.052      68.6    69.265    70.375    71.422    72.168   
   강북지역 NaN    68.209    69.016    69.636    71.052     72.26    73.204   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2008년 06월 2008년 07월 2008년 08월 2008년 09월 2008년 10월 2008년 11월  \
서울 NaN  NaN     73.33    73.928    74.419    75.041     75.47    75.027   
   강북지역 NaN    75.131    75.534    75.812    76.302    76.441    75.992   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2008년 12월 2009년 01월 2009년 02월 2009년 03월 2009년 04월 2009년 05월  \
서울 NaN  NaN    73.555    72.473    72.279    72.243    72.298     72.35   
   강북지역 NaN    75.025    74.197    74.074    73.947    74.225    74.268   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2009년 06월 2009년 07월 2009년 08월 2009년 09월 2009년 10월 2009년 11월  \
서울 NaN  NaN    72.568    72.978    73.495    74.899    75.903    76.242   
   강북지역 NaN     74.41    74.767    75.236    76.217    76.985    77.217   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -         -         -         -         -         -   

            2009년 12월 2010년 01월 2010년 02월 2010년 03월 2010년 04월 2010년 05월  \
서울 NaN  NaN    76.279    76.447    76.864    77.606    78.161    78.294   
   강북지역 NaN    77.215    77.416    77.849    78.746    79.432    79.535   
        도심권         -         -         -         -         -         -   
        동북권         -         -         -         -         -         -   
        서북권         -        

In [159]:
yunrip_lease_price_index_df = yunrip_lease_price_index_df.loc['서울']

In [160]:
yunrip_lease_price_index_df.head()

2005년 12월 2006년 01월 2006년 02월 2006년 03월 2006년 04월 2006년 05월  \
NaN  NaN    57.028    57.204    57.452    57.989    58.382    58.568   
강북지역 NaN    56.955     57.09    57.204    57.736    58.081    58.156   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2006년 06월 2006년 07월 2006년 08월 2006년 09월 2006년 10월 2006년 11월  \
NaN  NaN    58.806    58.897    59.194    59.883    60.731    61.629   
강북지역 NaN    58.312    58.375    58.814    59.494    60.813    61.656   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2006년 12월 2007년 01월 2007년 02월 2007년 03월 2007년 04월 2007년 05월  \
NaN  NaN    62.125    62.511     63.07    63.611    64.127    64.495   
강북지역 NaN    62.085    62.492    62.895     63.49    64.089     64.42   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2007년 06월 2007년 07월 2007년 08월 2007년 09월 2007년 10월 2007년 11월  \
NaN  NaN    64.742    65.289    65.583    66.628    67.292    67.727   
강북지역 NaN    64.554    65.162    65.494    66.473    67.046    67.743   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2007년 12월 2008년 01월 2008년 02월 2008년 03월 2008년 04월 2008년 05월  \
NaN  NaN    68.052      68.6    69.265    70.375    71.422    72.168   
강북지역 NaN    68.209    69.016    69.636    71.052     72.26    73.204   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2008년 06월 2008년 07월 2008년 08월 2008년 09월 2008년 10월 2008년 11월  \
NaN  NaN     73.33    73.928    74.419    75.041     75.47    75.027   
강북지역 NaN    75.131    75.534    75.812    76.302    76.441    75.992   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2008년 12월 2009년 01월 2009년 02월 2009년 03월 2009년 04월 2009년 05월  \
NaN  NaN    73.555    72.473    72.279    72.243    72.298     72.35   
강북지역 NaN    75.025    74.197    74.074    73.947    74.225    74.268   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2009년 06월 2009년 07월 2009년 08월 2009년 09월 2009년 10월 2009년 11월  \
NaN  NaN    72.568    72.978    73.495    74.899    75.903    76.242   
강북지역 NaN     74.41    74.767    75.236    76.217    76.985    77.217   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2009년 12월 2010년 01월 2010년 02월 2010년 03월 2010년 04월 2010년 05월  \
NaN  NaN    76.279    76.447    76.864    77.606    78.161    78.294   
강북지역 NaN    77.215    77.416    77.849    78.746    79.432    79.535   
     도심권         -         -         -         -         -         -   
     동북권         -         -         -         -         -         -   
     서북권         -         -         -         -         -         -   

         2010년 06월 2010년 07월 2010년 08월 2010년 09월 2010년 10월 2010년 11월  \
NaN  NaN    78.432    78.471    78.703  

In [161]:
yunrip_lease_price_index_df.index

MultiIndex([(   nan,   nan),
            ('강북지역',   nan),
            ('강북지역', '도심권'),
            ('강북지역', '동북권'),
            ('강북지역', '서북권'),
            ('강남지역', '서북권'),
            ('강남지역', '서남권'),
            ('강남지역', '동남권')],
           )

In [163]:
idx_list = []
for i in range(len(yunrip_lease_price_index_df.index)):
    if i == 0:
        idx_list.append('서울전체')
    elif i == 1:
        idx_list.append('강북전체')
    elif i == 5:
        idx_list.append('강남전체')
    else:
        idx_list.append(yunrip_lease_price_index_df.index[i][1])
idx_list

['서울전체', '강북전체', '도심권', '동북권', '서북권', '강남전체', '서남권', '동남권']

In [164]:
yunrip_lease_price_index_df.index = idx_list
yunrip_lease_price_index_df

,2005년 12월,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,57.028,57.204,57.452,57.989,58.382,58.568,58.806,58.897,59.194,59.883,60.731,61.629,62.125,62.511,63.07,63.611,64.127,64.495,64.742,65.289,65.583,66.628,67.292,67.727,68.052,68.6,69.265,70.375,71.422,72.168,73.33,73.928,74.419,75.041,75.47,75.027,73.555,72.473,72.279,72.243,72.298,72.35,72.568,72.978,73.495,74.899,75.903,76.242,76.279,76.447,76.864,77.606,78.161,78.294,78.432,78.471,78.703,79.306,79.904,80.315,80.63,81.312,82.324,83.087,83.634,83.964,84.326,84.779,86.058,86.928,87.685,87.744,87.675,87.690,87.948,88.096,88.063,87.916,87.932,87.844,87.819,88.253,88.787,89.004,89.207,89.374,89.674,90.116,90.345,90.497,90.590,90.919,91.321,91.811,92.173,92.516,92.662,92.893,93.232,93.391,93.391,93.429,93.455,93.477,93.541,93.685,93.829,93.998,94.078,94.269,94.474,94.696,94.918,95.170,95.492,95.764,95.982,96.288,96.618,97.038,97.242,97.359,97.398,97.464,97.571,97.670,97.766,97.856,97.937,98.036,98.172,98.255,98.328,98.378,98.416,98.535,98.707,98.941,99.122,99.327,99.462,99.639,99.806,100,100.130,100.291,100.471,100.477,100.447,100.420,100.408,100.484,100.623,100.761,100.868,100.885,100.850,100.738,100.610,100.490,100.397,100.339,100.277,100.246,100.265,100.290,100.391,100.505,100.652,100.741,100.816,100.889,100.905,100.932,100.975,101.091,101.272,101.461,101.681,101.914,102.164
강북전체,56.955,57.09,57.204,57.736,58.081,58.156,58.312,58.375,58.814,59.494,60.813,61.656,62.085,62.492,62.895,63.49,64.089,64.42,64.554,65.162,65.494,66.473,67.046,67.743,68.209,69.016,69.636,71.052,72.26,73.204,75.131,75.534,75.812,76.302,76.441,75.992,75.025,74.197,74.074,73.947,74.225,74.268,74.41,74.767,75.236,76.217,76.985,77.217,77.215,77.416,77.849,78.746,79.432,79.535,79.455,79.514,79.68,80.146,80.802,81.078,81.336,82.093,82.942,83.734,84.103,84.318,84.69,85.022,86.538,87.659,88.302,88.364,88.297,88.311,88.311,88.531,88.483,88.367,88.470,88.423,88.448,88.692,89.264,89.493,89.598,89.598,89.861,90.364,90.699,90.831,90.966,91.259,91.630,91.981,92.398,92.741,92.869,93.057,93.431,93.572,93.514,93.562,93.561,93.577,93.650,93.781,93.931,94.086,94.145,94.312,94.497,94.707,94.879,95.139,95.452,95.711,95.926,96.242,96.638,97.089,97.315,97.451,97.513,97.592,97.715,97.799,97.898,98.

In [166]:
yunrip_lease_df = main_data_df[(main_data_df['건물종류'] == '다세대연립')&(main_data_df['전월세매매구분'] == '전세')]
yunrip_lease_df.shape

(508977, 18)

In [167]:
yunrip_lease_df['구'] = yunrip_lease_df['지번주소'].apply(get_gu)
yunrip_lease_df['구'].value_counts()

송파구     60293
강서구     43493
은평구     36772
강동구     29880
양천구     29558
마포구     28515
서초구     26671
광진구     24784
동작구     23631
관악구     22689
강남구     22606
중랑구     16837
구로구     15910
용산구     15627
성북구     14850
서대문구    13240
강북구     13233
도봉구     11164
영등포구    10674
금천구     10341
동대문구     8998
노원구      8474
성동구      8060
종로구      7910
중구       4750
Name: 구, dtype: int64

In [168]:
for gu in tqdm(yunrip_lease_df['구'].unique()):
    if pd.isna(gu) == True:
        continue
    gu_df = yunrip_lease_df[yunrip_lease_df['구'] == gu]
    for col in yunrip_lease_price_index_df.columns:
        splitted = col.split(' ')
        year = int(splitted[0].replace('년',''))
        month = int(splitted[1].replace('월',''))
        date_df = gu_df[(gu_df['년'] == year)&(gu_df['월'] == month)]
        
        if gu in 도심권:
            area = '도심권'
        elif gu in 동북권:
            area = '동북권'
        elif gu in 서북권:
            area = '서북권'
        elif gu in 서남권:
            area = '서남권'
        elif gu in 동남권:
            area = '동남권'
        else:
            print(gu)
            raise ValueError('myerror')
        
        price_idx = yunrip_lease_price_index_df.loc[area, col]
        if price_idx == '-':
            if area in ['도심권', '동북권', '서북권']:
                price_idx = yunrip_lease_price_index_df.loc['강북전체', col]
            elif area in ['서남권', '동남권']:
                price_idx = yunrip_lease_price_index_df.loc['강남전체', col]
        
        date_df['주택가격지수'] = price_idx
        
        main_data_df.loc[date_df.index, '주택가격지수'] = date_df['주택가격지수']

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:10<00:00,  2.73s/it]


In [169]:
filenames

['월간_매매가격지수_아파트.xlsx',
 '월간_매매가격지수_연립다세대.xlsx',
 '월간_매매가격지수_종합.xlsx',
 '월간_전세가격지수_아파트.xlsx',
 '월간_전세가격지수_연립다세대.xlsx',
 '월간_전세가격지수_종합.xlsx']

In [171]:
gross_lease_price_index_df = pd.read_excel(basedir + filenames[5], header=[10,11], index_col=[0,1,2])
print(gross_lease_price_index_df.shape)
gross_lease_price_index_df.head()

(204, 360)


지 역        2006년 01월       2006년 02월       2006년 03월       2006년 04월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    54.828  0.62    55.226  0.73    55.902  1.22    56.421  0.93   
   종로구 NaN    68.261  1.12    69.091  1.22    69.785     1    69.878  0.13   
   중구  NaN    58.672 -0.12    59.337  1.13    59.739  0.68    60.264  0.88   
   용산구 NaN     63.82  0.56    64.064  0.38    64.303  0.37    66.347  3.18   
   성동구 NaN    55.484  0.15    55.546  0.11    56.523  1.76    57.048  0.93   

지 역        2006년 05월       2006년 06월       2006년 07월       2006년 08월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    56.706  0.51    56.857  0.27    56.983  0.22    57.198  0.38   
   종로구 NaN    70.306  0.61    70.244 -0.09    70.386   0.2    70.894  0.72   
   중구  NaN    60.544  0.47    61.055  0.84    61.069  0.02    61.476  0.67   
   용산구 NaN     66.82  0.71    66.904  0.13    66.939  0.05    66.895 -0.06   
   성동구 NaN    57.485  0.77     57.57  0.15    57.772  0.35    57.761 -0.02   

지 역        2006년 09월       2006년 10월       2006년 11월       2006년 12월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    57.934  1.29    58.769  1.44    59.699  1.58    60.104  0.68   
   종로구 NaN    71.207  0.44    71.642  0.61    72.158  0.72    72.299   0.2   
   중구  NaN    62.275   1.3    63.282  1.62    63.866  0.92    64.611  1.17   
   용산구 NaN    67.007  0.17    67.214  0.31    67.797  0.87    67.979  0.27   
   성동구 NaN    58.041  0.48    58.506   0.8    58.879  0.64    59.381  0.85   

지 역        2007년 01월       2007년 02월       2007년 03월       2007년 04월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    60.489  0.64     60.85   0.6    61.304  0.75    61.723  0.68   
   종로구 NaN    72.432  0.18    72.554  0.17    72.844   0.4    72.851  0.01   
   중구  NaN    64.745  0.21    65.049  0.47    65.126  0.12    65.163  0.06   
   용산구 NaN    68.108  0.19    68.056 -0.08    68.127   0.1    68.234  0.16   
   성동구 NaN    59.731  0.59    60.223  0.82    60.707   0.8     60.91  0.33   

지 역        2007년 05월       2007년 06월       2007년 07월       2007년 08월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    61.851  0.21    61.964  0.18    62.201  0.38    62.406  0.33   
   종로구 NaN    72.871  0.03    72.902  0.04    72.993  0.13    73.099  0.15   
   중구  NaN    65.436  0.42    65.595  0.24    65.592     0    65.592     0   
   용산구 NaN    68.223 -0.02    68.241  0.03     68.31   0.1    68.313     0   
   성동구 NaN    60.856 -0.09    60.823 -0.05    60.904  0.13     61.03  0.21   

지 역        2007년 09월       2007년 10월       2007년 11월       2007년 12월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    62.887  0.77    63.179  0.46    63.434   0.4    63.537  0.16   
   종로구 NaN    74.194   1.5    74.559  0.49    74.559     0    74.808  0.33   
   중구  NaN    66.161  0.87    66.406  0.37    66.544  0.21    66.759  0.32   
   용산구 NaN    68.355  0.06    68.307 -0.07    68.754  0.65    68.906  0.22   
   성동구 NaN    61.194  0.27     61.45  0.42    61.588  0.22    61.653  0.11   

지 역        2008년 01월       2008년 02월       2008년 03월       2008년 04월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    63.819  0.44    64.188  0.58    64.927  1.15    65.661  1.13   
   종로구 NaN     74.79 -0.02    75.363  0.77    77.116  2.33    78.006  1.15   
   중구  NaN    66.866  0.16    67.106  0.36    67.482  0.56    68.008  0.78   
   용산구 NaN    69.715  1.17    70.217  0.72    70.275  0.08    70.744  0.67   
   성동구 NaN    62.046  0.64    62.164  0.19    62.869  1.13    63.124   0.4   

지 역        2008년 05월       2008년 06월       2008년 07월       2008년 08월        \
                  지수   변동률        지수   변동률        지수   변동률        지수   변동률   
서울 NaN NaN    6

In [172]:
cols_to_drop = [col for col in gross_lease_price_index_df.columns if '변동률' in col]
cols_to_drop[:5]

[('2006년 01월', '변동률'),
 ('2006년 02월', '변동률'),
 ('2006년 03월', '변동률'),
 ('2006년 04월', '변동률'),
 ('2006년 05월', '변동률')]

In [ ]:
gross_lease_price_index_df.drop(columns=cols_to_drop, inplace=True)
gross_lease_price_index_df.columns = [col[0] for col in gross_lease_price_index_df.columns]
print(gross_lease_price_index_df.shape)
gross_lease_price_index_df

In [174]:
gross_lease_price_index_df = gross_lease_price_index_df.loc['서울']
gross_lease_price_index_df

,,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
NaN,NaN,54.828,55.226,55.902,56.421,56.706,56.857,56.983,57.198,57.934,58.769,59.699,60.104,60.489,60.85,61.304,61.723,61.851,61.964,62.201,62.406,62.887,63.179,63.434,63.537,63.819,64.188,64.927,65.661,66.075,66.617,66.917,67.085,67.423,67.456,66.917,65.383,64.481,64.468,64.667,64.9,65.106,65.379,65.818,66.335,67.821,68.544,68.956,69.142,69.42,69.902,70.426,70.788,70.954,71.082,71.146,71.321,71.779,72.428,72.984,73.402,74.066,75.259,76.274,76.892,76.917,77.26,77.887,78.997,80.226,80.923,81.071,80.993,80.938,80.985,80.99,80.761,80.501,80.304,80.018,79.867,80.268,80.809,81.189,81.361,81.602,81.969,82.436,82.688,82.835,82.942,83.379,84.04,84.807,85.692,86.306,86.71,87.415,87.972,88.415,88.48,88.478,88.49,88.585,88.704,88.978,89.312,89.564,89.788,90.045,90.464,91.009,91.737,92.291,92.934,93.428,93.862,94.491,95.104,95.814,96.296,96.519,96.656,96.764,96.89,97.036,97.211,97.398,97.528,97.654,97.848,98.061,98.175,98.231,98.27,98.402,98.576,98.81,99.096,99.313,99.511,99.645,99.786,100,100.169,100.367,100.539,100.454,100.249,100.01,99.872,99.927,100.126,100.382,100.556,100.552,100.421,100.097,99.669,99.282,99.074,98.973,98.921,98.918,99.003,99.104,99.334,99.598,99.973,100.402,100.564,100.682,100.749,100.796,100.944,101.237,101.676,102.088,102.442,102.98,103.628
종로구,NaN,68.261,69.091,69.785,69.878,70.306,70.244,70.386,70.894,71.207,71.642,72.158,72.299,72.432,72.554,72.844,72.851,72.871,72.902,72.993,73.099,74.194,74.559,74.559,74.808,74.79,75.363,77.116,78.006,78.491,78.504,78.571,78.569,78.96,79.641,78.603,78.077,77.205,77.49,77.711,77.794,77.798,77.912,77.892,78.576,79.744,80.183,80.382,79.663,80.222,80.243,80.421,80.425,80.457,80.427,80.619,80.792,81.005,81.599,81.653,81.933,82.311,82.863,83.375,83.641,84.422,84.581,84.731,85.015,85.963,87.101,87.583,87.628,87.628,87.628,87.628,87.685,87.685,87.82,87.834,87.857,87.923,87.983,88.004,88.053,88.074,88.094,88.396,88.876,89.12,89.434,89.669,89.995,90.407,90.719,90.88,90.926,91.163,91.602,91.731,91.822,91.828,91.831,91.811,91.84,91.961,92.094,92.136,92.161,92.407,92.497,92.794,93.167,93.416,93.953,94.162,94.269,94.598,94.944,95.395,95.504,95.607,95.685,95.709,95.809,95.937,96.127,96.363,96.569,96.772,97.058,97.286,97.48,

In [176]:
idx_list = []
for i in range(len(gross_lease_price_index_df.index)):
    if i == 0:
        idx_list.append('서울전체')
    else:
        idx_list.append(gross_lease_price_index_df.index[i][0])

idx_list

['서울전체',
 '종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [177]:
gross_lease_price_index_df.index = idx_list
gross_lease_price_index_df

,2006년 01월,2006년 02월,2006년 03월,2006년 04월,2006년 05월,2006년 06월,2006년 07월,2006년 08월,2006년 09월,2006년 10월,2006년 11월,2006년 12월,2007년 01월,2007년 02월,2007년 03월,2007년 04월,2007년 05월,2007년 06월,2007년 07월,2007년 08월,2007년 09월,2007년 10월,2007년 11월,2007년 12월,2008년 01월,2008년 02월,2008년 03월,2008년 04월,2008년 05월,2008년 06월,2008년 07월,2008년 08월,2008년 09월,2008년 10월,2008년 11월,2008년 12월,2009년 01월,2009년 02월,2009년 03월,2009년 04월,2009년 05월,2009년 06월,2009년 07월,2009년 08월,2009년 09월,2009년 10월,2009년 11월,2009년 12월,2010년 01월,2010년 02월,2010년 03월,2010년 04월,2010년 05월,2010년 06월,2010년 07월,2010년 08월,2010년 09월,2010년 10월,2010년 11월,2010년 12월,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,54.828,55.226,55.902,56.421,56.706,56.857,56.983,57.198,57.934,58.769,59.699,60.104,60.489,60.85,61.304,61.723,61.851,61.964,62.201,62.406,62.887,63.179,63.434,63.537,63.819,64.188,64.927,65.661,66.075,66.617,66.917,67.085,67.423,67.456,66.917,65.383,64.481,64.468,64.667,64.9,65.106,65.379,65.818,66.335,67.821,68.544,68.956,69.142,69.42,69.902,70.426,70.788,70.954,71.082,71.146,71.321,71.779,72.428,72.984,73.402,74.066,75.259,76.274,76.892,76.917,77.26,77.887,78.997,80.226,80.923,81.071,80.993,80.938,80.985,80.99,80.761,80.501,80.304,80.018,79.867,80.268,80.809,81.189,81.361,81.602,81.969,82.436,82.688,82.835,82.942,83.379,84.04,84.807,85.692,86.306,86.71,87.415,87.972,88.415,88.48,88.478,88.49,88.585,88.704,88.978,89.312,89.564,89.788,90.045,90.464,91.009,91.737,92.291,92.934,93.428,93.862,94.491,95.104,95.814,96.296,96.519,96.656,96.764,96.89,97.036,97.211,97.398,97.528,97.654,97.848,98.061,98.175,98.231,98.27,98.402,98.576,98.81,99.096,99.313,99.511,99.645,99.786,100,100.169,100.367,100.539,100.454,100.249,100.01,99.872,99.927,100.126,100.382,100.556,100.552,100.421,100.097,99.669,99.282,99.074,98.973,98.921,98.918,99.003,99.104,99.334,99.598,99.973,100.402,100.564,100.682,100.749,100.796,100.944,101.237,101.676,102.088,102.442,102.98,103.628
종로구,68.261,69.091,69.785,69.878,70.306,70.244,70.386,70.894,71.207,71.642,72.158,72.299,72.432,72.554,72.844,72.851,72.871,72.902,72.993,73.099,74.194,74.559,74.559,74.808,74.79,75.363,77.116,78.006,78.491,78.504,78.571,78.569,78.96,79.641,78.603,78.077,77.205,77.49,77.711,77.794,77.798,77.912,77.892,78.576,79.744,80.183,80.382,79.663,80.222,80.243,80.421,80.425,80.457,80.427,80.619,80.792,81.005,81.599,81.653,81.933,82.311,82.863,83.375,83.641,84.422,84.581,84.731,85.015,85.963,87.101,87.583,87.628,87.628,87.628,87.628,87.685,87.685,87.82,87.834,87.857,87.923,87.983,88.004,88.053,88.074,88.094,88.396,88.876,89.12,89.434,89.669,89.995,90.407,90.719,90.88,90.926,91.163,91.602,91.731,91.822,91.828,91.831,91.811,91.84,91.961,92.094,92.136,92.161,92.407,92.497,92.794,93.167,93.416,93.953,94.162,94.269,94.598,94.944,95.395,95.504,95.607,95.685,95.709,95.809,95.937,96.127,96.363,96.569,96.772,97.058,97.286,97.48,97.63,97

In [178]:
officetel_lease_df = main_data_df[(main_data_df['건물종류'] == '오피스텔')&(main_data_df['전월세매매구분'] == '전세')]
officetel_lease_df.shape

(121967, 18)

In [179]:
officetel_lease_df['구'] = officetel_lease_df['지번주소'].apply(get_gu)
officetel_lease_df['구'].value_counts()

강서구     19329
영등포구    18503
구로구      9129
마포구      8460
송파구      7392
강남구      7291
관악구      5980
은평구      5038
강동구      4589
양천구      4329
금천구      4124
동대문구     3719
서초구      3664
성동구      3346
용산구      2850
중구       2250
광진구      2150
서대문구     1709
도봉구      1596
종로구      1514
중랑구      1475
노원구      1247
성북구       953
동작구       807
강북구       515
Name: 구, dtype: int64

In [180]:
for gu in tqdm(officetel_lease_df['구'].unique()):
    if pd.isna(gu) == True:
        continue
    gu_df = officetel_lease_df[officetel_lease_df['구'] == gu]
    for col in gross_lease_price_index_df.columns:
        splitted = col.split(' ')
        year = int(splitted[0].replace('년',''))
        month = int(splitted[1].replace('월',''))
        date_df = gu_df[(gu_df['년'] == year)&(gu_df['월'] == month)]
        date_df['주택가격지수'] = gross_lease_price_index_df.loc[gu, col]
        main_data_df.loc[date_df.index, '주택가격지수'] = date_df['주택가격지수']

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:10<00:00,  2.72s/it]


In [182]:
main_data_df['주택가격지수'].isna().sum()

835206

In [ ]:
main_data_df.to_csv()

In [183]:
prac_df = main_data_df[(main_data_df['전월세매매구분']!='월세')]
prac_df['주택가격지수'].isna().sum()

1327

In [186]:
%%time
main_data_df.to_csv('./prepped_data/main_data_baseline_checkpoint_3.csv', index=False)

Wall time: 50.1 s
